In [1]:
import pandas as pd
import numpy as np 
from pandas import ExcelWriter
from pandas import ExcelFile

#the header must be 2 to account for the row index we want the header to start at
druginfo = pd.read_excel(r'C:\Users\danah\Downloads\C&SB M185 GBM Dataset.xlsx', sheet_name = 0, header=2) 

#make index of the database the patient barcodes
druginfo.index=druginfo['bcr_patient_barcode'] 

#drop the duplicate column & additional columns we don't want for analyis make this the database assigned to the variable name
druginfo.drop(['bcr_patient_barcode','bcr_drug_barcode', 'bcr_drug_uuid','drug_name','route_of_administration'], axis=1, inplace = True) 

druginfo.head() #show the header -- first few rows of the dataframe

#continue with same process as above, dropping specific columns
followup = pd.read_excel(r'C:\Users\danah\Downloads\C&SB M185 GBM Dataset.xlsx', sheet_name = 1, header = 2)
followup.index=followup['bcr_patient_barcode']
followup.drop(['bcr_patient_barcode', 'bcr_followup_barcode'], axis=1, inplace = True)


patinfo = pd.read_excel(r'C:\Users\danah\Downloads\C&SB M185 GBM Dataset.xlsx', sheet_name = 4, header = 2)
patinfo.index=patinfo['bcr_patient_barcode']
patinfo.drop(['bcr_patient_barcode'], axis=1, inplace = True)

radinfo = pd.read_excel(r'C:\Users\danah\Downloads\C&SB M185 GBM Dataset.xlsx', sheet_name = 5, header = 2)
radinfo.index=radinfo['bcr_patient_barcode']
radinfo.drop(['bcr_patient_barcode', 'bcr_radiation_barcode', 'units'], axis=1, inplace = True)
#importing of all the data from the online database


tumorvol = pd.read_excel(r'C:\Users\danah\Downloads\GBM Tumor Volume Data.xlsx')
tumorvol.index=tumorvol['bcr_patient_barcode']
tumorvol.drop(['bcr_patient_barcode'], axis = 1, inplace = True)

#importing of the tumor volume information obtained from the imaging available of TCIA




In [2]:
from collections import Counter #import this function in order to count all the Not Availables in the dataframes, they are strings making it difficult to use the .isna or .isnull functions


The next steps will involve identifying the columns within each database which do not meet our threshold of information and removing them. Using the Counter function with a for loop over the column dimensions to identify the percentage of each column that is Not Available thus identifying the appropriate indexes to drop. 

In [3]:
#determine the number of columns for each dataframe (except tumor vol), store in a variable
dshape = druginfo.shape 
fshape = followup.shape
pshape = patinfo.shape
rshape = radinfo.shape


In [4]:
#explanation of what is happening/syntax:
#create empty list to store percentages and for loop spanning number of columns
#use for loop index each column you want from the dataframe
#then turn that column into a list which allows the items from the dataframe to be pulled for checking
#Counter will then identify how many of those items are the search item - in this case Not Available
#divide the count by the number of rows - represents # of items checked, and multiple by 100 to make percentage
#append into empty list to store results for later analysis

druginfo_percNA = []
for i in range(dshape[1]):
    druginfo_percNA.append((Counter(druginfo[list(druginfo.columns)[i]])['[Not Available]']/dshape[0]))
print(druginfo_percNA)

[0.001455604075691412, 0.0, 0.056768558951965066, 0.12663755458515283, 0.1615720524017467, 0.9621542940320232, 0.4002911208151383, 0.19068413391557495, 0.1950509461426492, 0.039301310043668124, 0.2590975254730713]


In [5]:
finfo_percNA = []
for i in range(fshape[1]):
    finfo_percNA.append((Counter(followup[list(followup.columns)[i]])['[Not Available]']/fshape[0]))
print(finfo_percNA)

[0.0, 0.9252669039145908, 0.9537366548042705, 0.042704626334519574, 0.06405693950177936, 0.0, 0.09252669039145907, 0.0, 0.08540925266903915, 0.8825622775800712, 0.19217081850533807, 0.29537366548042704]


In [6]:
patinfo_percNA = []
for i in range(pshape[1]):
    patinfo_percNA.append((Counter(patinfo[list(patinfo.columns)[i]])['[Not Available]']/pshape[0]))
print(patinfo_percNA)

[0.0, 0.0, 0.0, 0.0, 0.034482758620689655, 0.18007662835249041, 0.0038314176245210726, 0.0, 0.0038314176245210726, 0.0, 0.038314176245210725, 0.0, 0.0842911877394636, 0.16475095785440613, 0.8735632183908046, 0.2720306513409962, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [7]:
radinfo_percNA = []
for i in range(rshape[1]):
    radinfo_percNA.append((Counter(radinfo[list(radinfo.columns)[i]])['[Not Available]']/rshape[0]))
print(radinfo_percNA)

[0.0, 0.01293103448275862, 0.017241379310344827, 0.07758620689655173, 0.1336206896551724, 0.017241379310344827, 0.05603448275862069, 0.004310344827586207, 0.021551724137931036]


In [8]:
threshold = 0.7
drug_cdrops = []
for i in range(len(druginfo_percNA)):
    if (druginfo_percNA[i] >= threshold):
        drug_cdrops.append(druginfo.columns[i]) #this identifies the name of the columns that need to be dropped by index
print(drug_cdrops) 

['measure_of_response']


In [9]:
f_cdrops = []
for i in range(len(finfo_percNA)):
    if (finfo_percNA[i] >= threshold):
        f_cdrops.append(followup.columns[i])
print(f_cdrops)

['followup_case_report_form_submission_reason', 'lost_follow_up', 'eastern_cancer_oncology_group']


In [10]:
pat_cdrops = []
for i in range(len(patinfo_percNA)):
    if (patinfo_percNA[i] >= threshold):
        pat_cdrops.append(patinfo.columns[i])
print(pat_cdrops)

['eastern_cancer_oncology_group']


In [11]:
rad_cdrops = []
for i in range(len(radinfo_percNA)):
    if (radinfo_percNA[i] >= threshold):
        rad_cdrops.append(radinfo.columns[i])
print(rad_cdrops) #this will have no columns dropped

[]


In [12]:
druginfo.drop(drug_cdrops, axis = 1, inplace = True) #test to see if 'measure of response' is removed before making it the actual dataframe
druginfo.head()

,form_completion_date,therapy_type,days_to_drug_therapy_start,therapy_ongoing,days_to_drug_therapy_end,number_cycles,prescribed_dose_units,total_dose_units,regimen_indication,total_dose
bcr_patient_barcode,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2793530,CDE_ID:3392465,CDE_ID:3103479,CDE_ID:3392470,CDE_ID:62590,CDE_ID:3065815,CDE_ID:3088785,CDE_ID:2793511,CDE_ID:1515
TCGA-02-0003,2009-4-2,Chemotherapy,49,NO,94,1,mg,mg,ADJUVANT,80
TCGA-02-0006,2009-3-30,Chemotherapy,88,[Not Available],148,7,mg/m2,mg/m2,ADJUVANT,200
TCGA-02-0006,2010-1-12,Chemotherapy,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],RECURRENCE,[Not Available]
TCGA-02-0006,2010-1-12,Chemotherapy,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],RECURRENCE,[Not Available]


In [13]:
followup.drop(f_cdrops, axis = 1, inplace = True)
followup.head()

,form_completion_date,radiation_therapy,postoperative_rx_tx,vital_status,days_to_last_followup,days_to_death,person_neoplasm_cancer_status,karnofsky_performance_score,performance_status_scale_timing
bcr_patient_barcode,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2005312,CDE_ID:3397567,CDE_ID:5,CDE_ID:3008273,CDE_ID:3165475,CDE_ID:2759550,CDE_ID:2003853,CDE_ID:2792763
TCGA-02-0003,2008-12-16,YES,YES,Dead,144,144,WITH TUMOR,100,[Not Available]
TCGA-02-0006,2008-12-17,YES,YES,Dead,558,558,WITH TUMOR,80,[Not Available]
TCGA-02-0009,2008-12-16,YES,[Not Available],Dead,322,322,WITH TUMOR,80,[Not Available]
TCGA-02-0011,2008-12-17,YES,YES,Dead,630,630,WITH TUMOR,80,[Not Available]


In [14]:
patinfo.drop(pat_cdrops, axis=1, inplace = True)

Now we must clean up the rows in order to prevent any patients who do not meet our threshold of information from continuing to the training of the Cox model. The threshold for the patients is 75% of the entries are present. 

Goal: Count the number of times the string Not available is present in each row. Remove any rows that are above the threshold, while maintaining the patients with images from our tumor volume dataset. 

In [15]:
#will not be able to use the counter function here, encountered KeyError every time I tried - instead use a pandas function 

search_string = "Not Available" #once again identifying any element containing this string


#(df.apply(lambda x: x.str.findall(search_string).str.len()).sum(axis=1).astype(int))
#explanation of the above code, which is what is used here:
#lambda function is used as a general/unnamed function to be applied to the dataframe's rows
#.str.findall() is identifying all the places our search string is located - essentially each time one is found, appended into an empty list
#.str.len() is used to count the number of times our search string is identified - counts how many strings are in result list
#sum the entire row's results as an integer


newd = pd.DataFrame(druginfo.apply(lambda x: x.str.findall(search_string).str.len()).sum(axis=1).astype(int)) #made a new dataframe to hold the results
newd.rename(columns = {0:"Missing Count"}, inplace = True) #results column was named missing count - index is still patient barcode
druginfo["Missing Count"] = list(newd["Missing Count"]) #added the column of missing count to druginfo dataframe
druginfo #checked to make sure this worked


,form_completion_date,therapy_type,days_to_drug_therapy_start,therapy_ongoing,days_to_drug_therapy_end,number_cycles,prescribed_dose_units,total_dose_units,regimen_indication,total_dose,Missing Count
bcr_patient_barcode,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2793530,CDE_ID:3392465,CDE_ID:3103479,CDE_ID:3392470,CDE_ID:62590,CDE_ID:3065815,CDE_ID:3088785,CDE_ID:2793511,CDE_ID:1515,0
TCGA-02-0003,2009-4-2,Chemotherapy,49,NO,94,1,mg,mg,ADJUVANT,80,0
TCGA-02-0006,2009-3-30,Chemotherapy,88,[Not Available],148,7,mg/m2,mg/m2,ADJUVANT,200,1
TCGA-02-0006,2010-1-12,Chemotherapy,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],RECURRENCE,[Not Available],7
TCGA-02-0006,2010-1-12,Chemotherapy,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],RECURRENCE,[Not Available],7
...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6662,2011-12-9,Chemotherapy,254,YES,[Not Available],04,mg,mg,PROGRESSION,1440,1
TCGA-76-6663,2011-11-15,Chemotherapy,44,YES,[Not Available],[Not Available],mg,mg,ADJUVANT,[Not Available],3
TCGA-76-6663,2011-11-15,Immunotherapy,93,NO,175,[Not Available],[Not Available],[Not Available],ADJUVANT,[Not Available],4


In [16]:
dthreshold = 0.75*(druginfo.shape[1]-1) #threshold is 50% of the columns in druginfo (subtract one because missing count isn't part of the original)

drug_filtered = druginfo[druginfo['Missing Count'] < dthreshold] #identifies rows that are below the threshold
drug_filtered #new druginfo dataframe with patients above the threshold removed

,form_completion_date,therapy_type,days_to_drug_therapy_start,therapy_ongoing,days_to_drug_therapy_end,number_cycles,prescribed_dose_units,total_dose_units,regimen_indication,total_dose,Missing Count
bcr_patient_barcode,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2793530,CDE_ID:3392465,CDE_ID:3103479,CDE_ID:3392470,CDE_ID:62590,CDE_ID:3065815,CDE_ID:3088785,CDE_ID:2793511,CDE_ID:1515,0
TCGA-02-0003,2009-4-2,Chemotherapy,49,NO,94,1,mg,mg,ADJUVANT,80,0
TCGA-02-0006,2009-3-30,Chemotherapy,88,[Not Available],148,7,mg/m2,mg/m2,ADJUVANT,200,1
TCGA-02-0006,2010-1-12,Chemotherapy,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],RECURRENCE,[Not Available],7
TCGA-02-0006,2010-1-12,Chemotherapy,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],RECURRENCE,[Not Available],7
...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6662,2011-12-9,Chemotherapy,254,YES,[Not Available],04,mg,mg,PROGRESSION,1440,1
TCGA-76-6663,2011-11-15,Chemotherapy,44,YES,[Not Available],[Not Available],mg,mg,ADJUVANT,[Not Available],3
TCGA-76-6663,2011-11-15,Immunotherapy,93,NO,175,[Not Available],[Not Available],[Not Available],ADJUVANT,[Not Available],4


In [17]:
newp = pd.DataFrame(patinfo.apply(lambda x: x.str.findall(search_string).str.len()).sum(axis=1).astype(int))
newp.rename(columns = {0:"Missing Count"}, inplace = True)
patinfo["Missing Count"] = list(newp["Missing Count"])
patinfo

,form_completion_date,prior_glioma,gender,days_to_birth,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status,...,person_neoplasm_cancer_status,karnofsky_performance_score,performance_status_scale_timing,age_at_initial_pathologic_diagnosis,days_to_initial_pathologic_diagnosis,histological_type,patient_id,tissue_source_site,tumor_tissue_site,Missing Count
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2786111,CDE_ID:2200604,CDE_ID:3008233,CDE_ID:2192199,CDE_ID:2192217,CDE_ID:3382737,CDE_ID:2896960,CDE_ID:2757941,CDE_ID:5,...,CDE_ID:2759550,CDE_ID:2003853,CDE_ID:2792763,CDE_ID:2006657,CDE_ID:3131740,CDE_ID:3081934,CDE_ID:,CDE_ID:,CDE_ID:3427536,0
TCGA-02-0003,2008-12-16,NO,MALE,-18341,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,...,WITH TUMOR,100,[Not Available],50,0,Untreated primary (de novo) GBM,0003,02,Brain,1
TCGA-02-0006,2008-12-17,NO,FEMALE,-20516,WHITE,NOT HISPANIC OR LATINO,No,2002,Tumor resection,Dead,...,WITH TUMOR,80,[Not Available],56,0,Untreated primary (de novo) GBM,0006,02,Brain,1
TCGA-02-0009,2008-12-16,NO,FEMALE,-22457,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,...,WITH TUMOR,80,[Not Available],61,0,Untreated primary (de novo) GBM,0009,02,Brain,1
TCGA-02-0011,2008-12-17,NO,FEMALE,-6926,WHITE,HISPANIC OR LATINO,No,2004,Tumor resection,Dead,...,WITH TUMOR,80,[Not Available],18,0,Untreated primary (de novo) GBM,0011,02,Brain,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6657,2011-11-15,NO,MALE,-27341,WHITE,NOT HISPANIC OR LATINO,No,2010,Tumor resection,Dead,...,WITH TUMOR,80,Preoperative,74,0,Untreated primary (de novo) GBM,6657,76,Brain,0
TCGA-76-6661,2011-11-15,NO,MALE,-19965,WHITE,[Not Available],No,2011,Tumor resection,Alive,...,WITH TUMOR,60,Preoperative,54,0,Untreated primary (de novo) GBM,6661,76,Brain,1
TCGA-76-6662,2011-12-9,NO,MALE,-21511,WHITE,NOT HISPANIC OR LATINO,No,2011,Tumor resection,Alive,...,WITH TUMOR,80,Post-Adjuvant Therapy,58,0,Untreated primary (de novo) GBM,6662,76,Brain,0


In [18]:
pthreshold = 0.75*(patinfo.shape[1]-1) #patinfo threshold

pat_filtered = patinfo[patinfo["Missing Count"] < pthreshold] #filtering for rows that meet the condition for the Missing Count column
pat_filtered #new patinfo dataframe with patients above the threshold removed

#NOTE: no patients are removed from this dataframe, therefore filtered and original would be the same

,form_completion_date,prior_glioma,gender,days_to_birth,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status,...,person_neoplasm_cancer_status,karnofsky_performance_score,performance_status_scale_timing,age_at_initial_pathologic_diagnosis,days_to_initial_pathologic_diagnosis,histological_type,patient_id,tissue_source_site,tumor_tissue_site,Missing Count
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2786111,CDE_ID:2200604,CDE_ID:3008233,CDE_ID:2192199,CDE_ID:2192217,CDE_ID:3382737,CDE_ID:2896960,CDE_ID:2757941,CDE_ID:5,...,CDE_ID:2759550,CDE_ID:2003853,CDE_ID:2792763,CDE_ID:2006657,CDE_ID:3131740,CDE_ID:3081934,CDE_ID:,CDE_ID:,CDE_ID:3427536,0
TCGA-02-0003,2008-12-16,NO,MALE,-18341,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,...,WITH TUMOR,100,[Not Available],50,0,Untreated primary (de novo) GBM,0003,02,Brain,1
TCGA-02-0006,2008-12-17,NO,FEMALE,-20516,WHITE,NOT HISPANIC OR LATINO,No,2002,Tumor resection,Dead,...,WITH TUMOR,80,[Not Available],56,0,Untreated primary (de novo) GBM,0006,02,Brain,1
TCGA-02-0009,2008-12-16,NO,FEMALE,-22457,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,...,WITH TUMOR,80,[Not Available],61,0,Untreated primary (de novo) GBM,0009,02,Brain,1
TCGA-02-0011,2008-12-17,NO,FEMALE,-6926,WHITE,HISPANIC OR LATINO,No,2004,Tumor resection,Dead,...,WITH TUMOR,80,[Not Available],18,0,Untreated primary (de novo) GBM,0011,02,Brain,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6657,2011-11-15,NO,MALE,-27341,WHITE,NOT HISPANIC OR LATINO,No,2010,Tumor resection,Dead,...,WITH TUMOR,80,Preoperative,74,0,Untreated primary (de novo) GBM,6657,76,Brain,0
TCGA-76-6661,2011-11-15,NO,MALE,-19965,WHITE,[Not Available],No,2011,Tumor resection,Alive,...,WITH TUMOR,60,Preoperative,54,0,Untreated primary (de novo) GBM,6661,76,Brain,1
TCGA-76-6662,2011-12-9,NO,MALE,-21511,WHITE,NOT HISPANIC OR LATINO,No,2011,Tumor resection,Alive,...,WITH TUMOR,80,Post-Adjuvant Therapy,58,0,Untreated primary (de novo) GBM,6662,76,Brain,0


In [19]:
newf = pd.DataFrame(followup.apply(lambda x: x.str.findall(search_string).str.len()).sum(axis=1).astype(int))
newf.rename(columns = {0:"Missing Count"}, inplace = True)
followup["Missing Count"] = list(newf["Missing Count"])
#followup1 -- include this to show the change in the dataframes
#note: once this cell is run and the new column is added -> cannot run and not have this error message without clearing the kernel

In [20]:
fthreshold = 0.75*(followup.shape[1]-1) #f threshold
f_filtered = followup[followup["Missing Count"] < fthreshold]
f_filtered #new followup dataframe with patients above the threshold removed

,form_completion_date,radiation_therapy,postoperative_rx_tx,vital_status,days_to_last_followup,days_to_death,person_neoplasm_cancer_status,karnofsky_performance_score,performance_status_scale_timing,Missing Count
bcr_patient_barcode,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2005312,CDE_ID:3397567,CDE_ID:5,CDE_ID:3008273,CDE_ID:3165475,CDE_ID:2759550,CDE_ID:2003853,CDE_ID:2792763,0
TCGA-02-0003,2008-12-16,YES,YES,Dead,144,144,WITH TUMOR,100,[Not Available],1
TCGA-02-0006,2008-12-17,YES,YES,Dead,558,558,WITH TUMOR,80,[Not Available],1
TCGA-02-0009,2008-12-16,YES,[Not Available],Dead,322,322,WITH TUMOR,80,[Not Available],2
TCGA-02-0011,2008-12-17,YES,YES,Dead,630,630,WITH TUMOR,80,[Not Available],1
...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6662,2015-3-26,[Not Available],[Not Available],Dead,[Not Available],1048,[Unknown],[Not Available],[Not Available],5
TCGA-76-6663,2011-11-15,YES,YES,Alive,235,[Not Applicable],WITH TUMOR,80,Preoperative,0
TCGA-76-6663,2015-3-26,[Not Available],[Not Available],Dead,[Not Available],567,[Unknown],[Not Available],[Not Available],5


In [21]:
newr = pd.DataFrame(radinfo.apply(lambda x: x.str.findall(search_string).str.len()).sum(axis=1).astype(int))
newr.rename(columns = {0:"Missing Count"}, inplace = True)
radinfo["Missing Count"] = list(newr["Missing Count"])
radinfo

,form_completion_date,radiation_type,anatomic_treatment_site,radiation_dosage,numfractions,days_to_radiation_therapy_start,radiation_treatment_ongoing,days_to_radiation_therapy_end,regimen_indication,Missing Count
bcr_patient_barcode,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2842944,CDE_ID:2793522,CDE_ID:2721441,CDE_ID:61465,CDE_ID:3008313,CDE_ID:2842745,CDE_ID:3008333,CDE_ID:2793511,0
TCGA-02-0003,2009-4-2,EXTERNAL BEAM,Primary Tumor Field,6000,30,49,NO,94,ADJUVANT,0
TCGA-02-0006,2009-3-30,EXTERNAL BEAM,Primary Tumor Field,6000,30,21,[Not Available],67,ADJUVANT,1
TCGA-02-0011,2009-4-2,EXTERNAL BEAM,Primary Tumor Field,6000,30,20,NO,61,ADJUVANT,0
TCGA-02-0027,2009-3-27,EXTERNAL BEAM,Primary Tumor Field,6000,50,27,NO,68,ADJUVANT,0
...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6656,2011-11-15,EXTERNAL BEAM,Primary Tumor Field,5940,[Not Available],42,NO,89,ADJUVANT,1
TCGA-76-6657,2011-11-15,EXTERNAL BEAM,Primary Tumor Field,60,30,48,NO,80,ADJUVANT,0
TCGA-76-6661,2011-11-15,EXTERNAL BEAM,Primary Tumor Field,[Not Available],[Not Available],48,NO,87,ADJUVANT,2


In [22]:
rthreshold = 0.75*(radinfo.shape[1]-1) #radinfo threshold

radfiltered = radinfo[radinfo['Missing Count'] < threshold]
radfiltered #new dataframe for radinfo
    

,form_completion_date,radiation_type,anatomic_treatment_site,radiation_dosage,numfractions,days_to_radiation_therapy_start,radiation_treatment_ongoing,days_to_radiation_therapy_end,regimen_indication,Missing Count
bcr_patient_barcode,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2842944,CDE_ID:2793522,CDE_ID:2721441,CDE_ID:61465,CDE_ID:3008313,CDE_ID:2842745,CDE_ID:3008333,CDE_ID:2793511,0
TCGA-02-0003,2009-4-2,EXTERNAL BEAM,Primary Tumor Field,6000,30,49,NO,94,ADJUVANT,0
TCGA-02-0011,2009-4-2,EXTERNAL BEAM,Primary Tumor Field,6000,30,20,NO,61,ADJUVANT,0
TCGA-02-0027,2009-3-27,EXTERNAL BEAM,Primary Tumor Field,6000,50,27,NO,68,ADJUVANT,0
TCGA-02-0034,2009-4-6,EXTERNAL BEAM,Primary Tumor Field,6000,30,23,NO,68,ADJUVANT,0
...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6286,2011-11-15,EXTERNAL BEAM,Primary Tumor Field,60,30,-278,NO,67,ADJUVANT,0
TCGA-76-6286,2011-11-15,RADIOISOTOPE,Primary Tumor Field,35,3.5,250,NO,250,PROGRESSION,0
TCGA-76-6657,2011-11-15,EXTERNAL BEAM,Primary Tumor Field,60,30,48,NO,80,ADJUVANT,0


I will now be determining if any patients are duplicates within the individual dataframes and reshape them to the smallest sample size at that point. This pool of individuals will be used in our first training cohort of the Cox regression model. Afterwards, we will then determine how the cox regression handles the information of tumor volume - which is gathered from only 14 patients. 

In [23]:
tumorpats = ['TCGA-06-0125', 'TCGA-06-0125', 'TCGA-06-0125', 'TCGA-06-0125',
'TCGA-06-0132', 'TCGA-06-0132', 'TCGA-06-0132', 'TCGA-06-0133',
       'TCGA-06-0133', 'TCGA-06-0138', 'TCGA-06-0138', 'TCGA-06-0138',
       'TCGA-06-0184', 'TCGA-06-0184', 'TCGA-06-0184', 'TCGA-06-0184',
       'TCGA-06-0185', 'TCGA-06-0185', 'TCGA-06-0185', 'TCGA-06-0188',
       'TCGA-06-0188', 'TCGA-06-0188', 'TCGA-06-0188', 'TCGA-06-1084',
       'TCGA-06-1084', 'TCGA-06-1084', 'TCGA-06-1084', 'TCGA-14-0789',
       'TCGA-14-0789', 'TCGA-14-0789', 'TCGA-14-0865', 'TCGA-14-0865',
       'TCGA-14-0865', 'TCGA-14-1037', 'TCGA-14-1037', 'TCGA-14-1452',
       'TCGA-14-1452', 'TCGA-14-1459', 'TCGA-14-1459', 'TCGA-14-1459',
       'TCGA-14-1823', 'TCGA-14-1823'] 
#using tumorvol.index, I retrieved the patient barcodes of the patients I would need to check for


In [24]:
#explanation of syntax:
#using the .index and the condition, I am checking to see if the barcode of the dataframe is contained within my tumorpats list
#any() lets me see if it's in any of the rows of my specified dataframe
#sum() is counting how many times the barcode is present

for i in range(len(tumorpats)):
    print(tumorpats[i], ((radinfo.index == tumorpats[i]).any()).sum()) #2 are missing

TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0132 0
TCGA-06-0132 0
TCGA-06-0132 0
TCGA-06-0133 1
TCGA-06-0133 1
TCGA-06-0138 0
TCGA-06-0138 0
TCGA-06-0138 0
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-14-0789 1
TCGA-14-0789 1
TCGA-14-0789 1
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-1037 1
TCGA-14-1037 1
TCGA-14-1452 1
TCGA-14-1452 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1823 1
TCGA-14-1823 1


In [25]:
for i in range(len(tumorpats)):
    print(tumorpats[i], ((patinfo.index == tumorpats[i]).any()).sum()) #all are accounted for

TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0132 1
TCGA-06-0132 1
TCGA-06-0132 1
TCGA-06-0133 1
TCGA-06-0133 1
TCGA-06-0138 1
TCGA-06-0138 1
TCGA-06-0138 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-14-0789 1
TCGA-14-0789 1
TCGA-14-0789 1
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-1037 1
TCGA-14-1037 1
TCGA-14-1452 1
TCGA-14-1452 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1823 1
TCGA-14-1823 1


In [26]:
for i in range(len(tumorpats)):
    print(tumorpats[i], ((druginfo.index == tumorpats[i]).any()).sum()) #2 are missing

TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0132 0
TCGA-06-0132 0
TCGA-06-0132 0
TCGA-06-0133 1
TCGA-06-0133 1
TCGA-06-0138 0
TCGA-06-0138 0
TCGA-06-0138 0
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-14-0789 1
TCGA-14-0789 1
TCGA-14-0789 1
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-1037 1
TCGA-14-1037 1
TCGA-14-1452 1
TCGA-14-1452 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1823 1
TCGA-14-1823 1


In [27]:
for i in range(len(tumorpats)):
    print(tumorpats[i], ((followup.index == tumorpats[i]).any()).sum()) #all are present

TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0132 1
TCGA-06-0132 1
TCGA-06-0132 1
TCGA-06-0133 1
TCGA-06-0133 1
TCGA-06-0138 1
TCGA-06-0138 1
TCGA-06-0138 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-14-0789 1
TCGA-14-0789 1
TCGA-14-0789 1
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-1037 1
TCGA-14-1037 1
TCGA-14-1452 1
TCGA-14-1452 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1823 1
TCGA-14-1823 1


In [28]:
patinfo = patinfo.loc[~patinfo.index.duplicated(keep = 'first')] #absolutely no duplicate information in this dataframe
patinfo

,form_completion_date,prior_glioma,gender,days_to_birth,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status,...,person_neoplasm_cancer_status,karnofsky_performance_score,performance_status_scale_timing,age_at_initial_pathologic_diagnosis,days_to_initial_pathologic_diagnosis,histological_type,patient_id,tissue_source_site,tumor_tissue_site,Missing Count
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2786111,CDE_ID:2200604,CDE_ID:3008233,CDE_ID:2192199,CDE_ID:2192217,CDE_ID:3382737,CDE_ID:2896960,CDE_ID:2757941,CDE_ID:5,...,CDE_ID:2759550,CDE_ID:2003853,CDE_ID:2792763,CDE_ID:2006657,CDE_ID:3131740,CDE_ID:3081934,CDE_ID:,CDE_ID:,CDE_ID:3427536,0
TCGA-02-0003,2008-12-16,NO,MALE,-18341,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,...,WITH TUMOR,100,[Not Available],50,0,Untreated primary (de novo) GBM,0003,02,Brain,1
TCGA-02-0006,2008-12-17,NO,FEMALE,-20516,WHITE,NOT HISPANIC OR LATINO,No,2002,Tumor resection,Dead,...,WITH TUMOR,80,[Not Available],56,0,Untreated primary (de novo) GBM,0006,02,Brain,1
TCGA-02-0009,2008-12-16,NO,FEMALE,-22457,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,...,WITH TUMOR,80,[Not Available],61,0,Untreated primary (de novo) GBM,0009,02,Brain,1
TCGA-02-0011,2008-12-17,NO,FEMALE,-6926,WHITE,HISPANIC OR LATINO,No,2004,Tumor resection,Dead,...,WITH TUMOR,80,[Not Available],18,0,Untreated primary (de novo) GBM,0011,02,Brain,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6657,2011-11-15,NO,MALE,-27341,WHITE,NOT HISPANIC OR LATINO,No,2010,Tumor resection,Dead,...,WITH TUMOR,80,Preoperative,74,0,Untreated primary (de novo) GBM,6657,76,Brain,0
TCGA-76-6661,2011-11-15,NO,MALE,-19965,WHITE,[Not Available],No,2011,Tumor resection,Alive,...,WITH TUMOR,60,Preoperative,54,0,Untreated primary (de novo) GBM,6661,76,Brain,1
TCGA-76-6662,2011-12-9,NO,MALE,-21511,WHITE,NOT HISPANIC OR LATINO,No,2011,Tumor resection,Alive,...,WITH TUMOR,80,Post-Adjuvant Therapy,58,0,Untreated primary (de novo) GBM,6662,76,Brain,0


In [29]:
druginfo = druginfo.loc[~druginfo.index.duplicated(keep='first')] #removal of duplicates brings us to a cohort of 204 patients 
druginfo

,form_completion_date,therapy_type,days_to_drug_therapy_start,therapy_ongoing,days_to_drug_therapy_end,number_cycles,prescribed_dose_units,total_dose_units,regimen_indication,total_dose,Missing Count
bcr_patient_barcode,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2793530,CDE_ID:3392465,CDE_ID:3103479,CDE_ID:3392470,CDE_ID:62590,CDE_ID:3065815,CDE_ID:3088785,CDE_ID:2793511,CDE_ID:1515,0
TCGA-02-0003,2009-4-2,Chemotherapy,49,NO,94,1,mg,mg,ADJUVANT,80,0
TCGA-02-0006,2009-3-30,Chemotherapy,88,[Not Available],148,7,mg/m2,mg/m2,ADJUVANT,200,1
TCGA-02-0011,2009-4-2,Chemotherapy,20,NO,145,4,mg/m2,mg/m2,"OTHER, SPECIFY IN NOTES",75,0
TCGA-02-0027,2009-3-27,Chemotherapy,27,[Not Available],256,6,mg/m2/day,mg/m2/day,ADJUVANT,1200,1
...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6657,2011-11-15,Chemotherapy,[Not Available],NO,[Not Available],[Not Available],mg,mg,ADJUVANT,[Not Available],4
TCGA-76-6661,2011-11-15,Chemotherapy,48,NO,115,[Not Available],mg,mg,ADJUVANT,[Not Available],2
TCGA-76-6662,2011-12-9,Chemotherapy,88,NO,227,[Not Available],mg,mg,ADJUVANT,[Not Available],2


In [30]:
followup = followup.loc[~followup.index.duplicated(keep='first')] #cohort is now 255 patients
followup

,form_completion_date,radiation_therapy,postoperative_rx_tx,vital_status,days_to_last_followup,days_to_death,person_neoplasm_cancer_status,karnofsky_performance_score,performance_status_scale_timing,Missing Count
bcr_patient_barcode,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2005312,CDE_ID:3397567,CDE_ID:5,CDE_ID:3008273,CDE_ID:3165475,CDE_ID:2759550,CDE_ID:2003853,CDE_ID:2792763,0
TCGA-02-0003,2008-12-16,YES,YES,Dead,144,144,WITH TUMOR,100,[Not Available],1
TCGA-02-0006,2008-12-17,YES,YES,Dead,558,558,WITH TUMOR,80,[Not Available],1
TCGA-02-0009,2008-12-16,YES,[Not Available],Dead,322,322,WITH TUMOR,80,[Not Available],2
TCGA-02-0011,2008-12-17,YES,YES,Dead,630,630,WITH TUMOR,80,[Not Available],1
...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6657,2011-11-15,YES,YES,Dead,153,153,WITH TUMOR,80,Preoperative,0
TCGA-76-6661,2011-11-15,YES,YES,Alive,7,[Not Applicable],WITH TUMOR,60,Preoperative,0
TCGA-76-6662,2011-12-9,YES,YES,Alive,282,[Not Applicable],WITH TUMOR,80,Post-Adjuvant Therapy,0


In [31]:
radinfo = radfiltered.loc[~radfiltered.index.duplicated(keep='first')] #cohort is now 157
radinfo

,form_completion_date,radiation_type,anatomic_treatment_site,radiation_dosage,numfractions,days_to_radiation_therapy_start,radiation_treatment_ongoing,days_to_radiation_therapy_end,regimen_indication,Missing Count
bcr_patient_barcode,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2842944,CDE_ID:2793522,CDE_ID:2721441,CDE_ID:61465,CDE_ID:3008313,CDE_ID:2842745,CDE_ID:3008333,CDE_ID:2793511,0
TCGA-02-0003,2009-4-2,EXTERNAL BEAM,Primary Tumor Field,6000,30,49,NO,94,ADJUVANT,0
TCGA-02-0011,2009-4-2,EXTERNAL BEAM,Primary Tumor Field,6000,30,20,NO,61,ADJUVANT,0
TCGA-02-0027,2009-3-27,EXTERNAL BEAM,Primary Tumor Field,6000,50,27,NO,68,ADJUVANT,0
TCGA-02-0034,2009-4-6,EXTERNAL BEAM,Primary Tumor Field,6000,30,23,NO,68,ADJUVANT,0
...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6280,2011-12-9,EXTERNAL BEAM,Primary Tumor Field,35,10,182,NO,192,PROGRESSION,0
TCGA-76-6286,2011-11-15,EXTERNAL BEAM,Primary Tumor Field,60,30,-278,NO,67,ADJUVANT,0
TCGA-76-6657,2011-11-15,EXTERNAL BEAM,Primary Tumor Field,60,30,48,NO,80,ADJUVANT,0


Now we must test to see if the 14 patients for the tumor images are within these reduced datasets

In [32]:
for i in range(len(tumorpats)):
    print(tumorpats[i], ((radinfo.index == tumorpats[i]).any()).sum()) #same 2 are missing

TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0132 0
TCGA-06-0132 0
TCGA-06-0132 0
TCGA-06-0133 1
TCGA-06-0133 1
TCGA-06-0138 0
TCGA-06-0138 0
TCGA-06-0138 0
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-14-0789 0
TCGA-14-0789 0
TCGA-14-0789 0
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-1037 1
TCGA-14-1037 1
TCGA-14-1452 1
TCGA-14-1452 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1823 1
TCGA-14-1823 1


In [33]:
for i in range(len(tumorpats)):
    print(tumorpats[i], ((druginfo.index == tumorpats[i]).any()).sum()) #same 2 are missing

TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0132 0
TCGA-06-0132 0
TCGA-06-0132 0
TCGA-06-0133 1
TCGA-06-0133 1
TCGA-06-0138 0
TCGA-06-0138 0
TCGA-06-0138 0
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-14-0789 1
TCGA-14-0789 1
TCGA-14-0789 1
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-1037 1
TCGA-14-1037 1
TCGA-14-1452 1
TCGA-14-1452 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1823 1
TCGA-14-1823 1


In [34]:
for i in range(len(tumorpats)):
    print(tumorpats[i], ((followup.index == tumorpats[i]).any()).sum()) #all are present

TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0132 1
TCGA-06-0132 1
TCGA-06-0132 1
TCGA-06-0133 1
TCGA-06-0133 1
TCGA-06-0138 1
TCGA-06-0138 1
TCGA-06-0138 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-14-0789 1
TCGA-14-0789 1
TCGA-14-0789 1
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-1037 1
TCGA-14-1037 1
TCGA-14-1452 1
TCGA-14-1452 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1823 1
TCGA-14-1823 1


In [35]:
for i in range(len(tumorpats)):
    print(tumorpats[i], ((patinfo.index == tumorpats[i]).any()).sum()) #all are present

TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0125 1
TCGA-06-0132 1
TCGA-06-0132 1
TCGA-06-0132 1
TCGA-06-0133 1
TCGA-06-0133 1
TCGA-06-0138 1
TCGA-06-0138 1
TCGA-06-0138 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0184 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0185 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-0188 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-06-1084 1
TCGA-14-0789 1
TCGA-14-0789 1
TCGA-14-0789 1
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-0865 1
TCGA-14-1037 1
TCGA-14-1037 1
TCGA-14-1452 1
TCGA-14-1452 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1459 1
TCGA-14-1823 1
TCGA-14-1823 1


In [36]:
threshold = 0.9*radinfo.shape[0]
list(radinfo.columns)

['form_completion_date',
 'radiation_type',
 'anatomic_treatment_site',
 'radiation_dosage',
 'numfractions',
 'days_to_radiation_therapy_start',
 'radiation_treatment_ongoing',
 'days_to_radiation_therapy_end',
 'regimen_indication',
 'Missing Count']

In [37]:
rlist = ['radiation_type',
 'anatomic_treatment_site',
 'radiation_dosage',
 'numfractions',
 'days_to_radiation_therapy_start',
 'radiation_treatment_ongoing',
 'days_to_radiation_therapy_end',
 'regimen_indication']
for i in rlist:
    if ((radinfo[i].str.findall(search_string).str.len()).sum()) > threshold:
        print(i, '\n')

In [38]:
patinfo.columns

Index(['form_completion_date', 'prior_glioma', 'gender', 'days_to_birth',
       'race', 'ethnicity', 'history_of_neoadjuvant_treatment',
       'year_of_initial_pathologic_diagnosis',
       'initial_pathologic_diagnosis_method', 'vital_status',
       'days_to_last_followup', 'days_to_death',
       'person_neoplasm_cancer_status', 'karnofsky_performance_score',
       'performance_status_scale_timing',
       'age_at_initial_pathologic_diagnosis',
       'days_to_initial_pathologic_diagnosis', 'histological_type',
       'patient_id', 'tissue_source_site', 'tumor_tissue_site',
       'Missing Count'],
      dtype='object')

In [39]:
pthreshold = 0.9*patinfo.shape[0]
plist = ['form_completion_date', 'prior_glioma', 'gender', 'days_to_birth',
       'race', 'ethnicity', 'history_of_neoadjuvant_treatment',
       'year_of_initial_pathologic_diagnosis',
       'initial_pathologic_diagnosis_method', 'vital_status',
       'days_to_last_followup', 'days_to_death',
       'person_neoplasm_cancer_status', 'karnofsky_performance_score',
       'performance_status_scale_timing',
       'age_at_initial_pathologic_diagnosis',
       'days_to_initial_pathologic_diagnosis', 'histological_type',
       'patient_id', 'tissue_source_site', 'tumor_tissue_site',]
for i in plist:
    if ((patinfo[i].str.findall(search_string).str.len()).sum()) > threshold:
        print(i, '\n')

In [40]:
druginfo.columns

Index(['form_completion_date', 'therapy_type', 'days_to_drug_therapy_start',
       'therapy_ongoing', 'days_to_drug_therapy_end', 'number_cycles',
       'prescribed_dose_units', 'total_dose_units', 'regimen_indication',
       'total_dose', 'Missing Count'],
      dtype='object')

In [41]:
dthreshold = 0.9*druginfo.shape[0]
dlist = ['form_completion_date', 'therapy_type', 'days_to_drug_therapy_start',
       'therapy_ongoing', 'days_to_drug_therapy_end', 'number_cycles',
       'prescribed_dose_units', 'total_dose_units', 'regimen_indication',
       'total_dose']
for i in dlist:
    if ((druginfo[i].str.findall(search_string).str.len()).sum()) > threshold:
        print(i, '\n')

In [42]:
followup.columns

Index(['form_completion_date', 'radiation_therapy', 'postoperative_rx_tx',
       'vital_status', 'days_to_last_followup', 'days_to_death',
       'person_neoplasm_cancer_status', 'karnofsky_performance_score',
       'performance_status_scale_timing', 'Missing Count'],
      dtype='object')

In [43]:
fthreshold = 0.9*followup.shape[0]
flist = ['form_completion_date', 'radiation_therapy', 'postoperative_rx_tx',
       'vital_status', 'days_to_last_followup', 'days_to_death',
       'person_neoplasm_cancer_status', 'karnofsky_performance_score',
       'performance_status_scale_timing']
for i in flist:
    if ((followup[i].str.findall(search_string).str.len()).sum()) > threshold:
        print(i, '\n')

2 of the tumor volume patients are no longer in 2 of the dataframes: considerations for how we would like to proceed regarding these patients must be made

Next steps may include: 
<br>a. Resizing the dataframes and making one large one
<br>b. using the patient information dataframe to examine how the Cox model handles a large sample size and look at the influence of those covariates
<br>c. combine the drug information and radiation information into one dataset with as they are the ones missing the 2 imaging patients, and specify other covariates from the other dataframes for the training
<br>d. run cox on all the dataframes to determine weight of each covariate before including the tumor images

In [44]:
testjoin1 = pd.merge(patinfo, followup, right_index = True, left_index = True, suffixes=('_pat', '_followup')) 
#merged the dataframes by the index, both indices had to be present for the merge to occur, hence any extras being dropped from the resulting dataframe

In [45]:
testjoin1

,form_completion_date_pat,prior_glioma,gender,days_to_birth,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status_pat,...,form_completion_date_followup,radiation_therapy,postoperative_rx_tx,vital_status_followup,days_to_last_followup_followup,days_to_death_followup,person_neoplasm_cancer_status_followup,karnofsky_performance_score_followup,performance_status_scale_timing_followup,Missing Count_followup
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2786111,CDE_ID:2200604,CDE_ID:3008233,CDE_ID:2192199,CDE_ID:2192217,CDE_ID:3382737,CDE_ID:2896960,CDE_ID:2757941,CDE_ID:5,...,CDE_ID:,CDE_ID:2005312,CDE_ID:3397567,CDE_ID:5,CDE_ID:3008273,CDE_ID:3165475,CDE_ID:2759550,CDE_ID:2003853,CDE_ID:2792763,0
TCGA-02-0003,2008-12-16,NO,MALE,-18341,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,...,2008-12-16,YES,YES,Dead,144,144,WITH TUMOR,100,[Not Available],1
TCGA-02-0006,2008-12-17,NO,FEMALE,-20516,WHITE,NOT HISPANIC OR LATINO,No,2002,Tumor resection,Dead,...,2008-12-17,YES,YES,Dead,558,558,WITH TUMOR,80,[Not Available],1
TCGA-02-0009,2008-12-16,NO,FEMALE,-22457,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,...,2008-12-16,YES,[Not Available],Dead,322,322,WITH TUMOR,80,[Not Available],2
TCGA-02-0011,2008-12-17,NO,FEMALE,-6926,WHITE,HISPANIC OR LATINO,No,2004,Tumor resection,Dead,...,2008-12-17,YES,YES,Dead,630,630,WITH TUMOR,80,[Not Available],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6657,2011-11-15,NO,MALE,-27341,WHITE,NOT HISPANIC OR LATINO,No,2010,Tumor resection,Dead,...,2011-11-15,YES,YES,Dead,153,153,WITH TUMOR,80,Preoperative,0
TCGA-76-6661,2011-11-15,NO,MALE,-19965,WHITE,[Not Available],No,2011,Tumor resection,Alive,...,2011-11-15,YES,YES,Alive,7,[Not Applicable],WITH TUMOR,60,Preoperative,0
TCGA-76-6662,2011-12-9,NO,MALE,-21511,WHITE,NOT HISPANIC OR LATINO,No,2011,Tumor resection,Alive,...,2011-12-9,YES,YES,Alive,282,[Not Applicable],WITH TUMOR,80,Post-Adjuvant Therapy,0


In [98]:
print(testjoin1.columns)

Index(['form_completion_date_pat', 'prior_glioma', 'gender', 'days_to_birth',
       'race', 'ethnicity', 'history_of_neoadjuvant_treatment',
       'year_of_initial_pathologic_diagnosis',
       'initial_pathologic_diagnosis_method', 'vital_status_pat',
       'days_to_last_followup_pat', 'days_to_death_pat',
       'person_neoplasm_cancer_status_pat', 'karnofsky_performance_score_pat',
       'performance_status_scale_timing_pat',
       'age_at_initial_pathologic_diagnosis',
       'days_to_initial_pathologic_diagnosis', 'histological_type',
       'patient_id', 'tissue_source_site', 'tumor_tissue_site',
       'Missing Count_pat', 'form_completion_date_followup',
       'radiation_therapy', 'postoperative_rx_tx', 'vital_status_followup',
       'days_to_last_followup_followup', 'days_to_death_followup',
       'person_neoplasm_cancer_status_followup',
       'karnofsky_performance_score_followup',
       'performance_status_scale_timing_followup', 'Missing Count_followup'],
      d

In [47]:
def checkduplicate(df1, df2):
    dfc1 = df1.columns.values.tolist()
    dfc2 = df2.columns.values.tolist()
    df1_columncheck = len(dfc1) - 1
    df2_columncheck = len(dfc2) -1
    for i in range(df1_columncheck):
        for j in range(df2_columncheck):
            if dfc1[i] == dfc2[j]:
                return print(dfc1[i],',', dfc2[j], 'are duplicates')

In [48]:
checkduplicate(followup, patinfo)


form_completion_date , form_completion_date are duplicates


In [49]:
duplicates1 = ['form_completion_date']

In [50]:
checkduplicate(followup, radinfo)

form_completion_date , form_completion_date are duplicates


In [51]:
checkduplicate(followup, druginfo)

form_completion_date , form_completion_date are duplicates


In [52]:
followup['form_completion_date']

bcr_patient_barcode
CDE_ID:2673794       CDE_ID:
TCGA-02-0003      2008-12-16
TCGA-02-0006      2008-12-17
TCGA-02-0009      2008-12-16
TCGA-02-0011      2008-12-17
                     ...    
TCGA-76-6657      2011-11-15
TCGA-76-6661      2011-11-15
TCGA-76-6662       2011-12-9
TCGA-76-6663      2011-11-15
TCGA-76-6664      2011-11-15
Name: form_completion_date, Length: 255, dtype: object

In [53]:
druginfo['form_completion_date'] #this was me double checking to see if the information was different in the column - it was

bcr_patient_barcode
CDE_ID:2673794       CDE_ID:
TCGA-02-0003        2009-4-2
TCGA-02-0006       2009-3-30
TCGA-02-0011        2009-4-2
TCGA-02-0027       2009-3-27
                     ...    
TCGA-76-6657      2011-11-15
TCGA-76-6661      2011-11-15
TCGA-76-6662       2011-12-9
TCGA-76-6663      2011-11-15
TCGA-76-6664      2011-11-15
Name: form_completion_date, Length: 204, dtype: object

In [54]:
checkduplicate(patinfo, druginfo)

form_completion_date , form_completion_date are duplicates


In [55]:
checkduplicate(patinfo, radinfo)

form_completion_date , form_completion_date are duplicates


In [56]:
checkduplicate(radinfo, druginfo)

form_completion_date , form_completion_date are duplicates


In [57]:
radinfo['form_completion_date']

bcr_patient_barcode
CDE_ID:2673794       CDE_ID:
TCGA-02-0003        2009-4-2
TCGA-02-0011        2009-4-2
TCGA-02-0027       2009-3-27
TCGA-02-0034        2009-4-6
                     ...    
TCGA-76-6280       2011-12-9
TCGA-76-6286      2011-11-15
TCGA-76-6657      2011-11-15
TCGA-76-6662       2011-12-9
TCGA-76-6663      2011-11-15
Name: form_completion_date, Length: 157, dtype: object

In [58]:
patinfo['form_completion_date'] #again checking to see what dataframes had the similar duplicate columns

bcr_patient_barcode
CDE_ID:2673794       CDE_ID:
TCGA-02-0003      2008-12-16
TCGA-02-0006      2008-12-17
TCGA-02-0009      2008-12-16
TCGA-02-0011      2008-12-17
                     ...    
TCGA-76-6657      2011-11-15
TCGA-76-6661      2011-11-15
TCGA-76-6662       2011-12-9
TCGA-76-6663      2011-11-15
TCGA-76-6664      2011-11-15
Name: form_completion_date, Length: 261, dtype: object

In [60]:
testjoin2 = pd.merge(radinfo, druginfo, right_index = True, left_index = True, suffixes = ('_rad', '_drug'))

In [61]:
testjoin2

,form_completion_date_rad,radiation_type,anatomic_treatment_site,radiation_dosage,numfractions,days_to_radiation_therapy_start,radiation_treatment_ongoing,days_to_radiation_therapy_end,regimen_indication_rad,Missing Count_rad,...,therapy_type,days_to_drug_therapy_start,therapy_ongoing,days_to_drug_therapy_end,number_cycles,prescribed_dose_units,total_dose_units,regimen_indication_drug,total_dose,Missing Count_drug
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2842944,CDE_ID:2793522,CDE_ID:2721441,CDE_ID:61465,CDE_ID:3008313,CDE_ID:2842745,CDE_ID:3008333,CDE_ID:2793511,0,...,CDE_ID:2793530,CDE_ID:3392465,CDE_ID:3103479,CDE_ID:3392470,CDE_ID:62590,CDE_ID:3065815,CDE_ID:3088785,CDE_ID:2793511,CDE_ID:1515,0
TCGA-02-0003,2009-4-2,EXTERNAL BEAM,Primary Tumor Field,6000,30,49,NO,94,ADJUVANT,0,...,Chemotherapy,49,NO,94,1,mg,mg,ADJUVANT,80,0
TCGA-02-0011,2009-4-2,EXTERNAL BEAM,Primary Tumor Field,6000,30,20,NO,61,ADJUVANT,0,...,Chemotherapy,20,NO,145,4,mg/m2,mg/m2,"OTHER, SPECIFY IN NOTES",75,0
TCGA-02-0027,2009-3-27,EXTERNAL BEAM,Primary Tumor Field,6000,50,27,NO,68,ADJUVANT,0,...,Chemotherapy,27,[Not Available],256,6,mg/m2/day,mg/m2/day,ADJUVANT,1200,1
TCGA-02-0034,2009-4-6,EXTERNAL BEAM,Primary Tumor Field,6000,30,23,NO,68,ADJUVANT,0,...,Chemotherapy,89,NO,159,2,mg/m2/day,mg/m2/day,ADJUVANT,200,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6280,2011-12-9,EXTERNAL BEAM,Primary Tumor Field,35,10,182,NO,192,PROGRESSION,0,...,Chemotherapy,27,NO,[Not Available],[Not Available],mg,mg,ADJUVANT,[Not Available],3
TCGA-76-6286,2011-11-15,EXTERNAL BEAM,Primary Tumor Field,60,30,-278,NO,67,ADJUVANT,0,...,Chemotherapy,510,NO,580,06,mg,mg,PROGRESSION,5940,0
TCGA-76-6657,2011-11-15,EXTERNAL BEAM,Primary Tumor Field,60,30,48,NO,80,ADJUVANT,0,...,Chemotherapy,[Not Available],NO,[Not Available],[Not Available],mg,mg,ADJUVANT,[Not Available],4


In [97]:
print(testjoin2.columns.values.tolist())

['form_completion_date_rad', 'radiation_type', 'anatomic_treatment_site', 'radiation_dosage', 'numfractions', 'days_to_radiation_therapy_start', 'radiation_treatment_ongoing', 'days_to_radiation_therapy_end', 'regimen_indication_rad', 'Missing Count_rad', 'form_completion_date_drug', 'therapy_type', 'days_to_drug_therapy_start', 'therapy_ongoing', 'days_to_drug_therapy_end', 'number_cycles', 'prescribed_dose_units', 'total_dose_units', 'regimen_indication_drug', 'total_dose', 'Missing Count_drug']


In [63]:
finaljoin = pd.merge(testjoin1, testjoin2, right_index = True, left_index = True)

In [64]:
finaljoin

,form_completion_date_pat,prior_glioma,gender,days_to_birth,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status_pat,...,therapy_type,days_to_drug_therapy_start,therapy_ongoing,days_to_drug_therapy_end,number_cycles,prescribed_dose_units,total_dose_units,regimen_indication_drug,total_dose,Missing Count_drug
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2786111,CDE_ID:2200604,CDE_ID:3008233,CDE_ID:2192199,CDE_ID:2192217,CDE_ID:3382737,CDE_ID:2896960,CDE_ID:2757941,CDE_ID:5,...,CDE_ID:2793530,CDE_ID:3392465,CDE_ID:3103479,CDE_ID:3392470,CDE_ID:62590,CDE_ID:3065815,CDE_ID:3088785,CDE_ID:2793511,CDE_ID:1515,0
TCGA-02-0003,2008-12-16,NO,MALE,-18341,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,...,Chemotherapy,49,NO,94,1,mg,mg,ADJUVANT,80,0
TCGA-02-0011,2008-12-17,NO,FEMALE,-6926,WHITE,HISPANIC OR LATINO,No,2004,Tumor resection,Dead,...,Chemotherapy,20,NO,145,4,mg/m2,mg/m2,"OTHER, SPECIFY IN NOTES",75,0
TCGA-02-0027,2008-12-17,NO,FEMALE,-12369,WHITE,NOT HISPANIC OR LATINO,No,2005,Tumor resection,Dead,...,Chemotherapy,27,[Not Available],256,6,mg/m2/day,mg/m2/day,ADJUVANT,1200,1
TCGA-02-0034,2008-12-17,NO,MALE,-22166,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,...,Chemotherapy,89,NO,159,2,mg/m2/day,mg/m2/day,ADJUVANT,200,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6280,2011-12-9,NO,MALE,-20931,WHITE,NOT HISPANIC OR LATINO,No,2007,Tumor resection,Dead,...,Chemotherapy,27,NO,[Not Available],[Not Available],mg,mg,ADJUVANT,[Not Available],3
TCGA-76-6286,2011-11-15,NO,MALE,-22173,WHITE,NOT HISPANIC OR LATINO,No,2009,Tumor resection,Dead,...,Chemotherapy,510,NO,580,06,mg,mg,PROGRESSION,5940,0
TCGA-76-6657,2011-11-15,NO,MALE,-27341,WHITE,NOT HISPANIC OR LATINO,No,2010,Tumor resection,Dead,...,Chemotherapy,[Not Available],NO,[Not Available],[Not Available],mg,mg,ADJUVANT,[Not Available],4


In [65]:
finaljoin.drop(['form_completion_date_followup', 'form_completion_date_rad', 'days_to_death_followup', "days_to_birth", 'days_to_initial_pathologic_diagnosis',  'prescribed_dose_units',
 'total_dose_units'], axis = 1, inplace = True)
finaljoin #dropped the columns 'form_completion_date_followup', 'form_completion_date_rad', 'days_to_death_followup', "days_to_birth", 'days_to_initial_pathologic_diagnosis',  'prescribed_dose_units', 'total_dose_units' due to repetition or unnecessary information

,form_completion_date_pat,prior_glioma,gender,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status_pat,days_to_last_followup_pat,...,Missing Count_rad,form_completion_date_drug,therapy_type,days_to_drug_therapy_start,therapy_ongoing,days_to_drug_therapy_end,number_cycles,regimen_indication_drug,total_dose,Missing Count_drug
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2786111,CDE_ID:2200604,CDE_ID:2192199,CDE_ID:2192217,CDE_ID:3382737,CDE_ID:2896960,CDE_ID:2757941,CDE_ID:5,CDE_ID:3008273,...,0,CDE_ID:,CDE_ID:2793530,CDE_ID:3392465,CDE_ID:3103479,CDE_ID:3392470,CDE_ID:62590,CDE_ID:2793511,CDE_ID:1515,0
TCGA-02-0003,2008-12-16,NO,MALE,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,144,...,0,2009-4-2,Chemotherapy,49,NO,94,1,ADJUVANT,80,0
TCGA-02-0011,2008-12-17,NO,FEMALE,WHITE,HISPANIC OR LATINO,No,2004,Tumor resection,Dead,630,...,0,2009-4-2,Chemotherapy,20,NO,145,4,"OTHER, SPECIFY IN NOTES",75,0
TCGA-02-0027,2008-12-17,NO,FEMALE,WHITE,NOT HISPANIC OR LATINO,No,2005,Tumor resection,Dead,315,...,0,2009-3-27,Chemotherapy,27,[Not Available],256,6,ADJUVANT,1200,1
TCGA-02-0034,2008-12-17,NO,MALE,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,430,...,0,2009-4-6,Chemotherapy,89,NO,159,2,ADJUVANT,200,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6280,2011-12-9,NO,MALE,WHITE,NOT HISPANIC OR LATINO,No,2007,Tumor resection,Dead,346,...,0,2011-12-9,Chemotherapy,27,NO,[Not Available],[Not Available],ADJUVANT,[Not Available],3
TCGA-76-6286,2011-11-15,NO,MALE,WHITE,NOT HISPANIC OR LATINO,No,2009,Tumor resection,Dead,638,...,0,2011-11-15,Chemotherapy,510,NO,580,06,PROGRESSION,5940,0
TCGA-76-6657,2011-11-15,NO,MALE,WHITE,NOT HISPANIC OR LATINO,No,2010,Tumor resection,Dead,153,...,0,2011-11-15,Chemotherapy,[Not Available],NO,[Not Available],[Not Available],ADJUVANT,[Not Available],4


In [96]:
print(finaljoin.columns.values.tolist()) #here is the list of all the current columns of information
#note - the suffixes at the end indicate which dataframe they originally come from 
#_drug = druginfo
#_pat = patinfo
#_rad = radinfo
#_followup = followup

['form_completion_date_pat', 'prior_glioma', 'gender', 'race', 'ethnicity', 'history_of_neoadjuvant_treatment', 'year_of_initial_pathologic_diagnosis', 'initial_pathologic_diagnosis_method', 'vital_status_pat', 'days_to_last_followup_pat', 'days_to_death_pat', 'person_neoplasm_cancer_status_pat', 'karnofsky_performance_score_pat', 'performance_status_scale_timing_pat', 'age_at_initial_pathologic_diagnosis', 'histological_type', 'patient_id', 'tissue_source_site', 'tumor_tissue_site', 'Missing Count_pat', 'radiation_therapy', 'postoperative_rx_tx', 'vital_status_followup', 'days_to_last_followup_followup', 'person_neoplasm_cancer_status_followup', 'karnofsky_performance_score_followup', 'performance_status_scale_timing_followup', 'Missing Count_followup', 'radiation_type', 'anatomic_treatment_site', 'radiation_dosage', 'numfractions', 'days_to_radiation_therapy_start', 'radiation_treatment_ongoing', 'days_to_radiation_therapy_end', 'regimen_indication_rad', 'Missing Count_rad', 'form_co

In [67]:
#once again checking for our tumor volume cohort

tumorpats = ['TCGA-06-0125',  'TCGA-06-0184', 'TCGA-06-0185', 'TCGA-06-0188', 'TCGA-06-1084','TCGA-14-0865','TCGA-14-0789', 'TCGA-14-1037','TCGA-14-1823'] 
for i in range(len(tumorpats)):
    print(tumorpats[i], ((finaljoin.index == tumorpats[i]).any()).sum())

#Note: the cohort is now 8 patients - dropped 06-0132, 06-0133, 06-0138, 14-1452, 14-1459    
#patient 14-0789 is now missing, was one of those dropped in the merging

TCGA-06-0125 1
TCGA-06-0184 1
TCGA-06-0185 1
TCGA-06-0188 1
TCGA-06-1084 1
TCGA-14-0865 1
TCGA-14-0789 0
TCGA-14-1037 1
TCGA-14-1823 1


In [68]:
#druginfo['drug_duration'] = druginfo['days_to_drug_therapy_end'] - druginfo['days_to_drug_therapy_start']
druginfo['days_to_drug_therapy_start'].iloc[0]

'CDE_ID:3392465'

In [69]:
drug_duration = [] #new column name for the drug duration
for i in range(druginfo.shape[0]):
    if (druginfo['days_to_drug_therapy_end'].iloc[i].isdigit() and druginfo['days_to_drug_therapy_start'].iloc[i].isdigit()) == True: #checks to see if the string is a digit
        drug_duration.append(int(druginfo['days_to_drug_therapy_end'].iloc[i]) - int(druginfo['days_to_drug_therapy_start'].iloc[i])) #finds difference between the columns (now integers) -> appends to new column
    else:
        drug_duration.append(np.nan) #appends nan to column if the string is not an integer
print(drug_duration)

[nan, 45, 60, 125, 229, 70, 45, 167, 335, 165, 930, 47, 20, 46, 72, 244, 161, 335, 5, 424, 43, 150, 106, 206, 39, 0, 75, 62, 26, 31, 0, 92, 61, 30, 61, 0, 46, 31, 94, 30, 7, 457, 638, 335, 658, 265, 337, 212, 61, 153, 31, 392, 237, nan, nan, 120, 151, 339, 38, 42, 183, 0, 92, nan, 41, 43, 449, 68, 176, 55, 160, 509, 62, 59, 33, 346, 124, 5, 4, 55, nan, 41, 55, nan, nan, 36, 153, nan, 4, 26, 99, 45, 62, 91, 41, 45, 48, 27, 212, 49, 75, 34, 39, 40, 127, 52, 112, 54, 86, 46, 110, 57, nan, nan, 31, nan, nan, 41, 15, 121, nan, 281, nan, nan, nan, 14, nan, nan, 14, nan, nan, 591, 1026, 73, nan, nan, nan, nan, 430, nan, nan, nan, nan, 246, 46, nan, 50, 43, 41, 13, 39, 59, 19, 43, 45, 6, 28, 18, nan, 31, 4, 15, 115, 44, 43, 19, 137, 15, 41, 78, 94, 188, 153, nan, 90, nan, 4, 87, 46, nan, 0, nan, nan, nan, nan, nan, 32, nan, nan, nan, nan, 154, nan, nan, nan, 189, nan, 70, nan, nan, 67, 139, nan, nan]


In [70]:
len(drug_duration) == druginfo.shape[0] #check that I have the correct amount in my string

True

In [71]:
druginfo.insert(druginfo.shape[1], 'drug_duration', drug_duration)

In [72]:
druginfo.replace({'[Not Available]': np.nan, '[Discrepancy]': np.nan}) #replaces all strings as nan -> which is an integer type

,form_completion_date,therapy_type,days_to_drug_therapy_start,therapy_ongoing,days_to_drug_therapy_end,number_cycles,prescribed_dose_units,total_dose_units,regimen_indication,total_dose,Missing Count,drug_duration
bcr_patient_barcode,,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2793530,CDE_ID:3392465,CDE_ID:3103479,CDE_ID:3392470,CDE_ID:62590,CDE_ID:3065815,CDE_ID:3088785,CDE_ID:2793511,CDE_ID:1515,0,NaN
TCGA-02-0003,2009-4-2,Chemotherapy,49,NO,94,1,mg,mg,ADJUVANT,80,0,45.0
TCGA-02-0006,2009-3-30,Chemotherapy,88,NaN,148,7,mg/m2,mg/m2,ADJUVANT,200,1,60.0
TCGA-02-0011,2009-4-2,Chemotherapy,20,NO,145,4,mg/m2,mg/m2,"OTHER, SPECIFY IN NOTES",75,0,125.0
TCGA-02-0027,2009-3-27,Chemotherapy,27,NaN,256,6,mg/m2/day,mg/m2/day,ADJUVANT,1200,1,229.0
...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6657,2011-11-15,Chemotherapy,NaN,NO,NaN,NaN,mg,mg,ADJUVANT,NaN,4,NaN
TCGA-76-6661,2011-11-15,Chemotherapy,48,NO,115,NaN,mg,mg,ADJUVANT,NaN,2,67.0
TCGA-76-6662,2011-12-9,Chemotherapy,88,NO,227,NaN,mg,mg,ADJUVANT,NaN,2,139.0


In [73]:
radinfo.head()

,form_completion_date,radiation_type,anatomic_treatment_site,radiation_dosage,numfractions,days_to_radiation_therapy_start,radiation_treatment_ongoing,days_to_radiation_therapy_end,regimen_indication,Missing Count
bcr_patient_barcode,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2842944,CDE_ID:2793522,CDE_ID:2721441,CDE_ID:61465,CDE_ID:3008313,CDE_ID:2842745,CDE_ID:3008333,CDE_ID:2793511,0
TCGA-02-0003,2009-4-2,EXTERNAL BEAM,Primary Tumor Field,6000,30,49,NO,94,ADJUVANT,0
TCGA-02-0011,2009-4-2,EXTERNAL BEAM,Primary Tumor Field,6000,30,20,NO,61,ADJUVANT,0
TCGA-02-0027,2009-3-27,EXTERNAL BEAM,Primary Tumor Field,6000,50,27,NO,68,ADJUVANT,0
TCGA-02-0034,2009-4-6,EXTERNAL BEAM,Primary Tumor Field,6000,30,23,NO,68,ADJUVANT,0


In [74]:
rad_duration = [] #new column name for the radiation duration
for i in range(radinfo.shape[0]):
    if (radinfo['days_to_radiation_therapy_end'].iloc[i].isdigit() and radinfo['days_to_radiation_therapy_start'].iloc[i].isdigit()) == True: #checks to see if the string is a digit
        rad_duration.append(int(radinfo['days_to_radiation_therapy_end'].iloc[i]) - int(radinfo['days_to_radiation_therapy_start'].iloc[i])) #finds difference between the columns (now integers) -> appends to new column
    else:
        rad_duration.append(np.nan) #appends nan to column if the string is not an integer
print(rad_duration)

[nan, 45, 41, 41, 45, 45, 48, 45, 55, 47, 47, 53, 46, 46, 48, 43, 3, 43, 39, 39, 39, 42, 52, 42, 0, 42, 60, 0, 7, 27, 43, 44, 14, 7, 43, 10, 25, 0, 44, 41, 42, 42, 41, 12, 9, 42, 34, 42, 44, 41, 43, 41, 39, 39, 42, 4, 42, 42, 48, 49, 42, 46, 0, 36, 43, 42, 46, 47, 45, 41, 50, 41, 48, 50, 49, 41, 47, 39, 41, 49, 51, 41, 40, 42, 53, 46, 44, 43, 39, 31, 39, 50, 24, 47, 3, 39, 43, 49, 42, 41, 44, 0, 42, 41, 58, 60, 44, 41, 0, 46, 31, 50, 43, 41, 41, 39, 42, 41, 43, 45, 48, 48, 43, 59, 46, 50, 8, 33, 44, 43, 43, 46, 50, 45, 4, 45, 31, 37, 45, 42, 4, 31, 44, 46, 31, 5, 4, 2, 33, 42, 62, 36, 10, nan, 32, 43, 42]


In [75]:
radinfo.insert(radinfo.shape[1], 'rad_duration', rad_duration) #insert the column at the end of the dataframe
radinfo.head() #check

,form_completion_date,radiation_type,anatomic_treatment_site,radiation_dosage,numfractions,days_to_radiation_therapy_start,radiation_treatment_ongoing,days_to_radiation_therapy_end,regimen_indication,Missing Count,rad_duration
bcr_patient_barcode,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2842944,CDE_ID:2793522,CDE_ID:2721441,CDE_ID:61465,CDE_ID:3008313,CDE_ID:2842745,CDE_ID:3008333,CDE_ID:2793511,0,NaN
TCGA-02-0003,2009-4-2,EXTERNAL BEAM,Primary Tumor Field,6000,30,49,NO,94,ADJUVANT,0,45.0
TCGA-02-0011,2009-4-2,EXTERNAL BEAM,Primary Tumor Field,6000,30,20,NO,61,ADJUVANT,0,41.0
TCGA-02-0027,2009-3-27,EXTERNAL BEAM,Primary Tumor Field,6000,50,27,NO,68,ADJUVANT,0,41.0
TCGA-02-0034,2009-4-6,EXTERNAL BEAM,Primary Tumor Field,6000,30,23,NO,68,ADJUVANT,0,45.0


In [76]:
testrad = radinfo.drop(['numfractions', 'days_to_radiation_therapy_start', 'days_to_radiation_therapy_end'], axis = 1)

In [77]:
testdrug = druginfo.drop(['days_to_drug_therapy_start', 'days_to_drug_therapy_end'], axis=1)

In [78]:
testjoin3 =  pd.merge(testrad, testdrug, right_index = True, left_index = True, suffixes = ("_r", "_d"))

In [79]:
testjoin3

,form_completion_date_r,radiation_type,anatomic_treatment_site,radiation_dosage,radiation_treatment_ongoing,regimen_indication_r,Missing Count_r,rad_duration,form_completion_date_d,therapy_type,therapy_ongoing,number_cycles,prescribed_dose_units,total_dose_units,regimen_indication_d,total_dose,Missing Count_d,drug_duration
bcr_patient_barcode,,,,,,,,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2842944,CDE_ID:2793522,CDE_ID:2721441,CDE_ID:2842745,CDE_ID:2793511,0,NaN,CDE_ID:,CDE_ID:2793530,CDE_ID:3103479,CDE_ID:62590,CDE_ID:3065815,CDE_ID:3088785,CDE_ID:2793511,CDE_ID:1515,0,NaN
TCGA-02-0003,2009-4-2,EXTERNAL BEAM,Primary Tumor Field,6000,NO,ADJUVANT,0,45.0,2009-4-2,Chemotherapy,NO,1,mg,mg,ADJUVANT,80,0,45.0
TCGA-02-0011,2009-4-2,EXTERNAL BEAM,Primary Tumor Field,6000,NO,ADJUVANT,0,41.0,2009-4-2,Chemotherapy,NO,4,mg/m2,mg/m2,"OTHER, SPECIFY IN NOTES",75,0,125.0
TCGA-02-0027,2009-3-27,EXTERNAL BEAM,Primary Tumor Field,6000,NO,ADJUVANT,0,41.0,2009-3-27,Chemotherapy,[Not Available],6,mg/m2/day,mg/m2/day,ADJUVANT,1200,1,229.0
TCGA-02-0034,2009-4-6,EXTERNAL BEAM,Primary Tumor Field,6000,NO,ADJUVANT,0,45.0,2009-4-6,Chemotherapy,NO,2,mg/m2/day,mg/m2/day,ADJUVANT,200,0,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6280,2011-12-9,EXTERNAL BEAM,Primary Tumor Field,35,NO,PROGRESSION,0,10.0,2011-12-9,Chemotherapy,NO,[Not Available],mg,mg,ADJUVANT,[Not Available],3,NaN
TCGA-76-6286,2011-11-15,EXTERNAL BEAM,Primary Tumor Field,60,NO,ADJUVANT,0,NaN,2011-11-15,Chemotherapy,NO,06,mg,mg,PROGRESSION,5940,0,70.0
TCGA-76-6657,2011-11-15,EXTERNAL BEAM,Primary Tumor Field,60,NO,ADJUVANT,0,32.0,2011-11-15,Chemotherapy,NO,[Not Available],mg,mg,ADJUVANT,[Not Available],4,NaN


In [80]:
testjoin3.drop('form_completion_date_d', axis=1, inplace = True)

In [95]:
print(patinfo.columns.values.tolist())

['form_completion_date', 'prior_glioma', 'gender', 'days_to_birth', 'race', 'ethnicity', 'history_of_neoadjuvant_treatment', 'year_of_initial_pathologic_diagnosis', 'initial_pathologic_diagnosis_method', 'vital_status', 'days_to_last_followup', 'days_to_death', 'person_neoplasm_cancer_status', 'karnofsky_performance_score', 'performance_status_scale_timing', 'age_at_initial_pathologic_diagnosis', 'days_to_initial_pathologic_diagnosis', 'histological_type', 'patient_id', 'tissue_source_site', 'tumor_tissue_site', 'Missing Count']


In [82]:
testpat = patinfo.drop(['patient_id', 'tumor_tissue_site', 'tissue_source_site'], axis = 1)

In [83]:
testfollow = followup.drop(['form_completion_date', 'vital_status', 'days_to_last_followup', 'days_to_death', 'person_neoplasm_cancer_status', 'karnofsky_performance_score', 'performance_status_scale_timing', 'Missing Count'], axis = 1)

In [84]:
testjoin4 = pd.merge(testpat, testfollow, left_index = True, right_index = True, suffixes = ('_p', '_f'))
testjoin4

,form_completion_date,prior_glioma,gender,days_to_birth,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status,...,days_to_death,person_neoplasm_cancer_status,karnofsky_performance_score,performance_status_scale_timing,age_at_initial_pathologic_diagnosis,days_to_initial_pathologic_diagnosis,histological_type,Missing Count,radiation_therapy,postoperative_rx_tx
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2786111,CDE_ID:2200604,CDE_ID:3008233,CDE_ID:2192199,CDE_ID:2192217,CDE_ID:3382737,CDE_ID:2896960,CDE_ID:2757941,CDE_ID:5,...,CDE_ID:3165475,CDE_ID:2759550,CDE_ID:2003853,CDE_ID:2792763,CDE_ID:2006657,CDE_ID:3131740,CDE_ID:3081934,0,CDE_ID:2005312,CDE_ID:3397567
TCGA-02-0003,2008-12-16,NO,MALE,-18341,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,...,144,WITH TUMOR,100,[Not Available],50,0,Untreated primary (de novo) GBM,1,YES,YES
TCGA-02-0006,2008-12-17,NO,FEMALE,-20516,WHITE,NOT HISPANIC OR LATINO,No,2002,Tumor resection,Dead,...,558,WITH TUMOR,80,[Not Available],56,0,Untreated primary (de novo) GBM,1,YES,YES
TCGA-02-0009,2008-12-16,NO,FEMALE,-22457,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,...,322,WITH TUMOR,80,[Not Available],61,0,Untreated primary (de novo) GBM,1,YES,[Not Available]
TCGA-02-0011,2008-12-17,NO,FEMALE,-6926,WHITE,HISPANIC OR LATINO,No,2004,Tumor resection,Dead,...,630,WITH TUMOR,80,[Not Available],18,0,Untreated primary (de novo) GBM,1,YES,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6657,2011-11-15,NO,MALE,-27341,WHITE,NOT HISPANIC OR LATINO,No,2010,Tumor resection,Dead,...,153,WITH TUMOR,80,Preoperative,74,0,Untreated primary (de novo) GBM,0,YES,YES
TCGA-76-6661,2011-11-15,NO,MALE,-19965,WHITE,[Not Available],No,2011,Tumor resection,Alive,...,[Not Applicable],WITH TUMOR,60,Preoperative,54,0,Untreated primary (de novo) GBM,1,YES,YES
TCGA-76-6662,2011-12-9,NO,MALE,-21511,WHITE,NOT HISPANIC OR LATINO,No,2011,Tumor resection,Alive,...,[Not Applicable],WITH TUMOR,80,Post-Adjuvant Therapy,58,0,Untreated primary (de novo) GBM,0,YES,YES


In [85]:
finaljoin1 = pd.merge(testjoin4, testjoin3, left_index = True, right_index = True) #merged the new dataframes once I dropped more columns and combined some of the information

In [86]:
finaljoin1

,form_completion_date,prior_glioma,gender,days_to_birth,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status,...,rad_duration,therapy_type,therapy_ongoing,number_cycles,prescribed_dose_units,total_dose_units,regimen_indication_d,total_dose,Missing Count_d,drug_duration
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2786111,CDE_ID:2200604,CDE_ID:3008233,CDE_ID:2192199,CDE_ID:2192217,CDE_ID:3382737,CDE_ID:2896960,CDE_ID:2757941,CDE_ID:5,...,NaN,CDE_ID:2793530,CDE_ID:3103479,CDE_ID:62590,CDE_ID:3065815,CDE_ID:3088785,CDE_ID:2793511,CDE_ID:1515,0,NaN
TCGA-02-0003,2008-12-16,NO,MALE,-18341,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,...,45.0,Chemotherapy,NO,1,mg,mg,ADJUVANT,80,0,45.0
TCGA-02-0011,2008-12-17,NO,FEMALE,-6926,WHITE,HISPANIC OR LATINO,No,2004,Tumor resection,Dead,...,41.0,Chemotherapy,NO,4,mg/m2,mg/m2,"OTHER, SPECIFY IN NOTES",75,0,125.0
TCGA-02-0027,2008-12-17,NO,FEMALE,-12369,WHITE,NOT HISPANIC OR LATINO,No,2005,Tumor resection,Dead,...,41.0,Chemotherapy,[Not Available],6,mg/m2/day,mg/m2/day,ADJUVANT,1200,1,229.0
TCGA-02-0034,2008-12-17,NO,MALE,-22166,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,...,45.0,Chemotherapy,NO,2,mg/m2/day,mg/m2/day,ADJUVANT,200,0,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6280,2011-12-9,NO,MALE,-20931,WHITE,NOT HISPANIC OR LATINO,No,2007,Tumor resection,Dead,...,10.0,Chemotherapy,NO,[Not Available],mg,mg,ADJUVANT,[Not Available],3,NaN
TCGA-76-6286,2011-11-15,NO,MALE,-22173,WHITE,NOT HISPANIC OR LATINO,No,2009,Tumor resection,Dead,...,NaN,Chemotherapy,NO,06,mg,mg,PROGRESSION,5940,0,70.0
TCGA-76-6657,2011-11-15,NO,MALE,-27341,WHITE,NOT HISPANIC OR LATINO,No,2010,Tumor resection,Dead,...,32.0,Chemotherapy,NO,[Not Available],mg,mg,ADJUVANT,[Not Available],4,NaN


In [87]:
print(finaljoin1.columns.values.tolist())

['form_completion_date', 'prior_glioma', 'gender', 'days_to_birth', 'race', 'ethnicity', 'history_of_neoadjuvant_treatment', 'year_of_initial_pathologic_diagnosis', 'initial_pathologic_diagnosis_method', 'vital_status', 'days_to_last_followup', 'days_to_death', 'person_neoplasm_cancer_status', 'karnofsky_performance_score', 'performance_status_scale_timing', 'age_at_initial_pathologic_diagnosis', 'days_to_initial_pathologic_diagnosis', 'histological_type', 'Missing Count', 'radiation_therapy', 'postoperative_rx_tx', 'form_completion_date_r', 'radiation_type', 'anatomic_treatment_site', 'radiation_dosage', 'radiation_treatment_ongoing', 'regimen_indication_r', 'Missing Count_r', 'rad_duration', 'therapy_type', 'therapy_ongoing', 'number_cycles', 'prescribed_dose_units', 'total_dose_units', 'regimen_indication_d', 'total_dose', 'Missing Count_d', 'drug_duration']


In [88]:
finaljoin1.drop(["days_to_birth", 'days_to_initial_pathologic_diagnosis',  'prescribed_dose_units', 'total_dose_units' ], axis = 1) #just getting an estimate on columns if I were to drop these

,form_completion_date,prior_glioma,gender,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status,days_to_last_followup,...,regimen_indication_r,Missing Count_r,rad_duration,therapy_type,therapy_ongoing,number_cycles,regimen_indication_d,total_dose,Missing Count_d,drug_duration
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
CDE_ID:2673794,CDE_ID:,CDE_ID:2786111,CDE_ID:2200604,CDE_ID:2192199,CDE_ID:2192217,CDE_ID:3382737,CDE_ID:2896960,CDE_ID:2757941,CDE_ID:5,CDE_ID:3008273,...,CDE_ID:2793511,0,NaN,CDE_ID:2793530,CDE_ID:3103479,CDE_ID:62590,CDE_ID:2793511,CDE_ID:1515,0,NaN
TCGA-02-0003,2008-12-16,NO,MALE,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,144,...,ADJUVANT,0,45.0,Chemotherapy,NO,1,ADJUVANT,80,0,45.0
TCGA-02-0011,2008-12-17,NO,FEMALE,WHITE,HISPANIC OR LATINO,No,2004,Tumor resection,Dead,630,...,ADJUVANT,0,41.0,Chemotherapy,NO,4,"OTHER, SPECIFY IN NOTES",75,0,125.0
TCGA-02-0027,2008-12-17,NO,FEMALE,WHITE,NOT HISPANIC OR LATINO,No,2005,Tumor resection,Dead,315,...,ADJUVANT,0,41.0,Chemotherapy,[Not Available],6,ADJUVANT,1200,1,229.0
TCGA-02-0034,2008-12-17,NO,MALE,WHITE,NOT HISPANIC OR LATINO,No,2003,Tumor resection,Dead,430,...,ADJUVANT,0,45.0,Chemotherapy,NO,2,ADJUVANT,200,0,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6280,2011-12-9,NO,MALE,WHITE,NOT HISPANIC OR LATINO,No,2007,Tumor resection,Dead,346,...,PROGRESSION,0,10.0,Chemotherapy,NO,[Not Available],ADJUVANT,[Not Available],3,NaN
TCGA-76-6286,2011-11-15,NO,MALE,WHITE,NOT HISPANIC OR LATINO,No,2009,Tumor resection,Dead,638,...,ADJUVANT,0,NaN,Chemotherapy,NO,06,PROGRESSION,5940,0,70.0
TCGA-76-6657,2011-11-15,NO,MALE,WHITE,NOT HISPANIC OR LATINO,No,2010,Tumor resection,Dead,153,...,ADJUVANT,0,32.0,Chemotherapy,NO,[Not Available],ADJUVANT,[Not Available],4,NaN


In [89]:
v1 = finaljoin1.loc[:, ['vital_status']].where(finaljoin1 == 'Dead') #using where turns the alive individuals into NaN
print(v1, v1['vital_status'].isnull().astype(int).sum()) #summing up the amount of nulls tells us how many people are alive

#40 individuals are alive

                    vital_status
bcr_patient_barcode             
CDE_ID:2673794               NaN
TCGA-02-0003                Dead
TCGA-02-0011                Dead
TCGA-02-0027                Dead
TCGA-02-0034                Dead
...                          ...
TCGA-76-6280                Dead
TCGA-76-6286                Dead
TCGA-76-6657                Dead
TCGA-76-6662                 NaN
TCGA-76-6663                 NaN

[150 rows x 1 columns] 41


C:\Users\danah\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


The following code is used to identify the patients who are either dead or alive and then puts either the days to last followup or the days to their death in the column for event_days -> will be used for Cox and Kaplan-Meier analysis. 

Know that there are 40 patients who remained alive therefore, the rest should identical to the information in the days to death column. 

In [107]:
#use where to output dataframe with the 'Not Applicable as nan'
a1 = finaljoin1.loc[:, ['days_to_last_followup']].where(finaljoin1 != '[Not Applicable]')
a2 = finaljoin1.loc[:, ['days_to_death']].where(finaljoin1 != '[Not Applicable]')

#put the values of the column values to lists
a1c = a1['days_to_last_followup'].values.tolist()
a2c = a2['days_to_death'].values.tolist()

#identify where the null values are located in the column
a1n = a1['days_to_last_followup'].isnull()
a2n = a2['days_to_death'].isnull()

#make new list to create column
event_days = []

#create for loop over length of dataframe/columns to iterate over each element -> each condition is identifying which element to put into the column and converting them to int at the same time
for i in range(len(a1c)):
    if a1c[i] == a2c[i]: 
        if a1c[i].isdigit() == True:
            event_days.append(int(a1c[i]))
    elif a1c[i] != a2c[i]:
        if a1n[i] == True and a2n[i] == False:
            if a2c[i].isdigit() == True:
                event_days.append(int(a2c[i]))
        elif a1n[i] == False and a2n[i] == True:
            if a1c[i].isdigit() == True:
                event_days.append(int(a1c[i]))
        elif a1n[i] or a2n[i] == False:
            if a1c[i].isdigit() == True:
                event_days.append(int(a1c[i]))
            elif a2c[i].isdigit() == True:
                event_days.append(int(a2c[i]))
    elif a1n[i] and a2n[i] == True:
        event_days.append(np.nan)
        
print(event_days, len(event_days), pd.Series(event_days).isnull().astype(int).sum())

#4 missing in a1c, 40 missing in a2c -> results now have none missing, checked with third term in the print statement

[144, 630, 315, 430, 199, 183, 600, 804, 873, 762, 634, 1561, 822, 355, 1489, 1439, 109, 691, 989, 428, 280, 508, 389, 161, 579, 67, 83, 1562, 1642, 1228, 1126, 866, 557, 684, 548, 455, 375, 178, 226, 263, 240, 285, 342, 268, 155, 237, 151, 690, 316, 104, 1637, 195, 447, 1084, 352, 107, 327, 146, 555, 527, 439, 181, 378, 1615, 1311, 277, 626, 658, 346, 277, 442, 121, 104, 287, 769, 414, 195, 1315, 784, 333, 460, 189, 419, 502, 801, 880, 485, 587, 24, 975, 216, 918, 1246, 187, 378, 30, 541, 543, 232, 218, 242, 115, 358, 666, 262, 434, 103, 327, 276, 181, 394, 141, 385, 112, 99, 294, 229, 737, 214, 343, 244, 242, 198, 164, 161, 165, 146, 505, 555, 1008, 648, 914, 427, 350, 753, 316, 316, 489, 231, 438, 480, 1458, 508, 100, 346, 638, 153, 282, 235] 149 0


In [91]:
testfinal = finaljoin1.replace({'[Not Available]': np.nan, '[Discrepancy]': np.nan, '[Not Applicable]':None})

In [110]:
finaldf = testfinal.drop(["days_to_birth", 'days_to_initial_pathologic_diagnosis',  'prescribed_dose_units', 'total_dose_units'], axis = 1) 
finaldf.drop('CDE_ID:2673794', axis = 0, inplace = True)

This should be the finaldf. It must now be binarized and split into testing/training data sets. The following code below works through this portion of the cleaning. 

In [105]:
finaldf.loc['TCGA-02-0068', ['vital_status','days_to_last_followup','days_to_death']]

vital_status             Dead
days_to_last_followup     804
days_to_death             804
Name: TCGA-02-0068, dtype: object

In [ ]:
finaldf['event_days'] = event_days #need to add the column for event days (identifies whether a patient was alive or dead) for Cox analysis

In [114]:
final_edit = finaldf

final_edit['vital_status'] =  np.where(final_edit['vital_status']=='Dead', 1, 0) #CURRENTLY SET TO 1 = DEAD, 0 = NOT (AKA ALIVE)
final_edit['prior_glioma'] =  np.where(final_edit['prior_glioma']=='YES', 1, 0)
final_edit['gender'] =  np.where(final_edit['gender']=='FEMALE', 1, 0)
final_edit['race'] =  np.where(final_edit['race']=='WHITE', 1, 0)
final_edit['ethnicity'] =  np.where(final_edit['ethnicity']=='HISPANIC OR LATINO', 1, 0)
final_edit['history_of_neoadjuvant_treatment'] =  np.where(final_edit['history_of_neoadjuvant_treatment']=='Yes', 1, 0)
# I AM UNSURE ABOUT INITIAL_PATH (below)
final_edit['initial_pathologic_diagnosis_method'] =  np.where(final_edit['initial_pathologic_diagnosis_method']=='Tumor resection', 1, 0)
# do we need this one?
final_edit['person_neoplasm_cancer_status'] =  np.where(final_edit['person_neoplasm_cancer_status']=='WITH TUMOR', 1, 0)
#final_edit['performance_status_scale_timing'] =  np.where(final_edit['performance_status_scale_timing']=='Post-Adjuvant Therapy' | final_edit['performance_status_scale_timing']=='Pre-Adjuvant Therapy', 1, 0)
conditions = [
    final_edit['performance_status_scale_timing'].eq('Post-Adjuvant Therapy'),
    final_edit['performance_status_scale_timing'].eq('Pre-Adjuvant Therapy') ]
final_edit['performance_status'] = np.select(conditions, [1,1], default=0)
del final_edit['performance_status_scale_timing']
# need this?
final_edit['histological_type'] =  np.where(final_edit['histological_type']=='Treated primary GBM', 1, 0)
final_edit['radiation_therapy'] =  np.where(final_edit['radiation_therapy']=='YES', 1, 0)
final_edit['postoperative_rx_tx'] =  np.where(final_edit['postoperative_rx_tx']=='YES', 1, 0)
final_edit['radiation_type'] =  np.where(final_edit['radiation_type']=='EXTERNAL BEAM', 1, 0)
# needed...?
final_edit['anatomic_treatment_site'] =  np.where(final_edit['anatomic_treatment_site']=='Primary Tumor Field', 1, 0)
final_edit['radiation_treatment_ongoing'] =  np.where(final_edit['radiation_treatment_ongoing']=='YES', 1, 0)
# DIFFERENCE BETWEEN THIS AND _D...?
final_edit['regimen_indication_r'] =  np.where(final_edit['regimen_indication_r']=='ADJUVANT', 1, 0)
final_edit['therapy_type'] =  np.where(final_edit['therapy_type']=='Chemotherapy', 1, 0)
final_edit['therapy_ongoing'] =  np.where(final_edit['therapy_ongoing']=='YES', 1, 0)
final_edit['regimen_indication_d'] =  np.where(final_edit['regimen_indication_d']=='ADJUVANT', 1, 0)

In [115]:
final_edit

,form_completion_date,prior_glioma,gender,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status,days_to_last_followup,...,rad_duration,therapy_type,therapy_ongoing,number_cycles,regimen_indication_d,total_dose,Missing Count_d,drug_duration,event_days,performance_status
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0003,2008-12-16,0,0,1,0,0,2003,1,1,144,...,45.0,1,0,1,1,80,0,45.0,144,0
TCGA-02-0011,2008-12-17,0,1,1,1,0,2004,1,1,630,...,41.0,1,0,4,0,75,0,125.0,630,0
TCGA-02-0027,2008-12-17,0,1,1,0,0,2005,1,1,315,...,41.0,1,0,6,1,1200,1,229.0,315,0
TCGA-02-0034,2008-12-17,0,0,1,0,0,2003,1,1,430,...,45.0,1,0,2,1,200,0,70.0,430,0
TCGA-02-0054,2008-12-16,0,1,1,0,0,2005,1,1,199,...,45.0,1,0,1,1,75,0,45.0,199,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-76-6280,2011-12-9,0,0,1,0,0,2007,1,1,346,...,10.0,1,0,NaN,1,NaN,3,NaN,346,1
TCGA-76-6286,2011-11-15,0,0,1,0,0,2009,1,1,638,...,NaN,1,0,06,0,5940,0,70.0,638,0
TCGA-76-6657,2011-11-15,0,0,1,0,0,2010,1,1,153,...,32.0,1,0,NaN,1,NaN,4,NaN,153,0


In [128]:
tumorpats = ['TCGA-06-0125',  'TCGA-06-0184', 'TCGA-06-0185', 'TCGA-06-0188', 'TCGA-06-1084','TCGA-14-0865', 'TCGA-14-1037','TCGA-14-1823'] 
newtumor = pd.DataFrame(columns = final_edit.columns.values.tolist())
for i in tumorpats:
    newtumor = newtumor.append(final_edit.loc[i])
newtumor

,form_completion_date,prior_glioma,gender,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status,days_to_last_followup,...,rad_duration,therapy_type,therapy_ongoing,number_cycles,regimen_indication_d,total_dose,Missing Count_d,drug_duration,event_days,performance_status
TCGA-06-0125,2008-12-8,0,1,1,0,0,2001,1,1,1439,...,3.0,1,0,14,1,300,0,424.0,1439,0
TCGA-06-0184,2008-12-10,0,0,1,0,0,2005,1,0,1228,...,14.0,1,0,12,0,160,0,335.0,1228,0
TCGA-06-0185,2008-12-8,0,0,1,0,0,2005,1,0,1126,...,7.0,0,0,24,0,689,0,658.0,1126,0
TCGA-06-0188,2008-12-9,0,0,1,0,0,2006,1,0,866,...,43.0,1,0,08,0,254,0,265.0,866,0
TCGA-06-1084,2009-5-11,0,0,0,0,0,2008,1,0,226,...,41.0,1,0,04,1,500,0,120.0,226,0
TCGA-14-0865,2009-8-4,0,0,1,0,0,2002,1,1,502,...,31.0,1,0,04,1,9500,0,121.0,502,1
TCGA-14-1037,2009-9-24,0,1,1,0,0,2002,1,1,587,...,47.0,0,0,NaN,1,NaN,5,NaN,587,1
TCGA-14-1823,2009-9-1,1,1,1,0,0,2007,1,1,543,...,41.0,1,0,12,1,11150,0,430.0,543,1


In [129]:
final_edit.to_csv(r'C:\Users\danah\Documents\Official Final Data with NaN and Event Days.csv')
newtumor.to_csv(r'C:\Users\danah\Documents\Final Tumor without Tumor Vol.csv')

In [135]:
finaltrainindices = np.random.choice(final_edit.index.values.tolist(), 100, replace = False) #another method for pulling sample
tumortrain = np.random.choice(newtumor.index.values.tolist(), 5, replace = False)

I didn't use the above method because it was not ideal for identifying the rows that matched the original dataframe, hence the following method being used. 

In [224]:
fulltrain = final_edit.sample(frac = .67, axis = 'index', replace = False) #pull random patients based on index and no replacement
fulltest = final_edit[~final_edit.index.isin(fulltest.index)].dropna() #identify rows that are in the sample dataframe and then drop them -> this produced an issue because it wasn't the right size
fulltest

,form_completion_date,prior_glioma,gender,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status,days_to_last_followup,...,rad_duration,therapy_type,therapy_ongoing,number_cycles,regimen_indication_d,total_dose,Missing Count_d,drug_duration,event_days,performance_status
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0011,2008-12-17,0,1,1,1,0,2004,1,1,630,...,41.0,1,0,4,0,75,0,125.0,630,0
TCGA-02-0054,2008-12-16,0,1,1,0,0,2005,1,1,199,...,45.0,1,0,1,1,75,0,45.0,199,0
TCGA-02-0060,2008-12-16,0,1,0,0,0,2006,1,1,183,...,48.0,1,0,3,1,75,0,167.0,183,0
TCGA-02-0064,2008-12-16,0,0,1,0,0,2005,1,1,600,...,45.0,1,0,12,1,200,0,335.0,600,0
TCGA-02-0075,2008-12-16,0,0,1,0,0,2005,1,1,634,...,53.0,1,0,1,1,75,0,20.0,634,0
TCGA-02-0102,2008-12-16,0,0,1,0,1,2004,1,1,822,...,46.0,1,0,5,1,1.4,0,244.0,822,0
TCGA-02-0106,2008-12-16,0,0,1,0,0,2004,1,1,355,...,48.0,1,0,5,1,500,0,161.0,355,0
TCGA-02-0116,2008-12-16,0,0,1,0,1,2003,1,1,1489,...,43.0,1,0,12,1,900,0,335.0,1489,0
TCGA-06-0125,2008-12-8,0,1,1,0,0,2001,1,1,1439,...,3.0,1,0,14,1,300,0,424.0,1439,0


In [196]:
tumortrain = newtumor.sample(n = 5, axis = 'index', replace = False) #pull random patients based on index and no replacement
tumortest = newtumor.isin(tumortrain) #check to see if the random samples are in original 
tumortest = tumortest.where(tumortest == False).dropna() #drop any rows from the samples in order to create other dataframe

In [203]:
fulltest.to_csv(r'C:\Users\danah\Documents\Full Cox Testing Set.csv') #exported for analysis once I checked output

In [201]:
fulltrain.to_csv(r'C:\Users\danah\Documents\Full Cox Training Set.csv') #exported for analysis once I checked output

In [204]:
tumortest.to_csv(r'C:\Users\danah\Documents\Tumor Testing Set.csv') #exported for analysis once I checked output

In [202]:
tumortrain.to_csv(r'C:\Users\danah\Documents\Tumor Training Set.csv') #exported for analysis once I checked output

In [205]:
fulltrain

,form_completion_date,prior_glioma,gender,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status,days_to_last_followup,...,rad_duration,therapy_type,therapy_ongoing,number_cycles,regimen_indication_d,total_dose,Missing Count_d,drug_duration,event_days,performance_status
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-02-0011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-02-0027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-02-0054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-02-0064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-27-2528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-76-6191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-76-6192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [206]:
fulltest

,form_completion_date,prior_glioma,gender,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status,days_to_last_followup,...,rad_duration,therapy_type,therapy_ongoing,number_cycles,regimen_indication_d,total_dose,Missing Count_d,drug_duration,event_days,performance_status
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
TCGA-12-1094,2009-3-17,0,0,1,0,0,2006,0,1,346,...,49.0,1,0,01,1,75,1,49.0,346,1
TCGA-08-0522,2008-12-11,0,0,1,0,0,2002,1,1,555,...,43.0,1,0,4,1,280,0,153.0,555,0
TCGA-76-4932,2011-4-6,0,1,1,0,0,2006,1,1,1458,...,42.0,1,0,NaN,0,NaN,6,NaN,1458,0
TCGA-06-0176,2008-12-10,0,0,0,0,0,2004,1,0,1562,...,43.0,1,0,12,0,400,0,457.0,1562,0
TCGA-19-5958,2011-6-3,0,0,1,0,0,2010,1,0,164,...,46.0,1,0,6,1,11500,0,137.0,164,1
TCGA-12-0769,2008-12-10,0,0,1,0,0,2002,0,1,378,...,45.0,1,0,2,1,200,0,45.0,378,1
TCGA-08-0350,2008-12-19,0,0,1,0,0,2001,1,1,104,...,39.0,0,0,NaN,0,450,1,160.0,104,1
TCGA-06-1801,2009-8-19,0,1,1,0,0,2008,1,0,263,...,12.0,1,0,05,1,280,1,151.0,263,0
TCGA-19-2629,2010-5-12,0,0,1,0,0,2009,1,1,NaN,...,59.0,1,0,2,1,3500,0,31.0,737,0


In [207]:
fulltrain = final_edit.apply(lambda x: x fulltest)

,form_completion_date,prior_glioma,gender,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status,days_to_last_followup,...,rad_duration,therapy_type,therapy_ongoing,number_cycles,regimen_indication_d,total_dose,Missing Count_d,drug_duration,event_days,performance_status
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0003,2008-12-16,0,0,1,0,0,2003,1,1,144,...,45.0,1,0,1,1,80,0,45.0,144,0
TCGA-02-0011,2008-12-17,0,1,1,1,0,2004,1,1,630,...,41.0,1,0,4,0,75,0,125.0,630,0
TCGA-02-0027,2008-12-17,0,1,1,0,0,2005,1,1,315,...,41.0,1,0,6,1,1200,1,229.0,315,0
TCGA-02-0054,2008-12-16,0,1,1,0,0,2005,1,1,199,...,45.0,1,0,1,1,75,0,45.0,199,0
TCGA-02-0064,2008-12-16,0,0,1,0,0,2005,1,1,600,...,45.0,1,0,12,1,200,0,335.0,600,0
TCGA-02-0068,2008-12-16,0,0,1,0,0,2006,1,1,804,...,55.0,1,0,3,1,75,0,165.0,804,0
TCGA-02-0075,2008-12-16,0,0,1,0,0,2005,1,1,634,...,53.0,1,0,1,1,75,0,20.0,634,0
TCGA-02-0085,2008-12-16,0,1,1,0,0,2003,1,1,1561,...,46.0,1,0,1,1,75,0,46.0,1561,0
TCGA-02-0102,2008-12-16,0,0,1,0,1,2004,1,1,822,...,46.0,1,0,5,1,1.4,0,244.0,822,0


In [220]:
fulltrain = final_edit[~final_edit.isin(fulltest)] #identify rows that are in the sample dataframe
fulltrain = fulltrain.where(fulltrain != np.nan).dropna()
fulltrain

,form_completion_date,prior_glioma,gender,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status,days_to_last_followup,...,rad_duration,therapy_type,therapy_ongoing,number_cycles,regimen_indication_d,total_dose,Missing Count_d,drug_duration,event_days,performance_status
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0003,2008-12-16,0.0,0.0,1.0,0.0,0.0,2003,1.0,1.0,144,...,45.0,1.0,0.0,1,1.0,80,0.0,45.0,144.0,0.0
TCGA-02-0011,2008-12-17,0.0,1.0,1.0,1.0,0.0,2004,1.0,1.0,630,...,41.0,1.0,0.0,4,0.0,75,0.0,125.0,630.0,0.0
TCGA-02-0027,2008-12-17,0.0,1.0,1.0,0.0,0.0,2005,1.0,1.0,315,...,41.0,1.0,0.0,6,1.0,1200,1.0,229.0,315.0,0.0
TCGA-02-0054,2008-12-16,0.0,1.0,1.0,0.0,0.0,2005,1.0,1.0,199,...,45.0,1.0,0.0,1,1.0,75,0.0,45.0,199.0,0.0
TCGA-02-0064,2008-12-16,0.0,0.0,1.0,0.0,0.0,2005,1.0,1.0,600,...,45.0,1.0,0.0,12,1.0,200,0.0,335.0,600.0,0.0
TCGA-02-0068,2008-12-16,0.0,0.0,1.0,0.0,0.0,2006,1.0,1.0,804,...,55.0,1.0,0.0,3,1.0,75,0.0,165.0,804.0,0.0
TCGA-02-0075,2008-12-16,0.0,0.0,1.0,0.0,0.0,2005,1.0,1.0,634,...,53.0,1.0,0.0,1,1.0,75,0.0,20.0,634.0,0.0
TCGA-02-0085,2008-12-16,0.0,1.0,1.0,0.0,0.0,2003,1.0,1.0,1561,...,46.0,1.0,0.0,1,1.0,75,0.0,46.0,1561.0,0.0
TCGA-02-0102,2008-12-16,0.0,0.0,1.0,0.0,1.0,2004,1.0,1.0,822,...,46.0,1.0,0.0,5,1.0,1.4,0.0,244.0,822.0,0.0


The above was methods tested to try and split the data into training and testing data sets. I determined the following below would be the best method upon further research. 

In [227]:
from sklearn.model_selection import train_test_split

fulltrain, fulltest = train_test_split(final_edit, test_size=0.3)

In [228]:
fulltrain

,form_completion_date,prior_glioma,gender,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status,days_to_last_followup,...,rad_duration,therapy_type,therapy_ongoing,number_cycles,regimen_indication_d,total_dose,Missing Count_d,drug_duration,event_days,performance_status
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,
TCGA-27-2518,2010-3-12,0,0,1,0,0,2007,1,1,753,...,44.0,1,0,03,1,500,0,87.0,753,0
TCGA-06-0168,2008-12-10,0,1,1,0,0,2002,1,1,579,...,0.0,1,0,02,0,380,0,31.0,579,0
TCGA-06-1802,2009-8-19,0,0,1,0,0,2008,1,0,240,...,9.0,1,0,11,0,845,0,339.0,240,0
TCGA-08-0522,2008-12-11,0,0,1,0,0,2002,1,1,555,...,43.0,1,0,4,1,280,0,153.0,555,0
TCGA-08-0353,2008-12-11,0,0,1,0,0,2003,1,1,195,...,4.0,1,0,2,1,200,0,62.0,195,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-02-0027,2008-12-17,0,1,1,0,0,2005,1,1,315,...,41.0,1,0,6,1,1200,1,229.0,315,0
TCGA-06-0192,2009-2-3,0,0,1,0,0,2007,1,0,557,...,10.0,0,0,10,0,873,0,337.0,557,0
TCGA-19-1787,2010-8-31,0,0,1,0,0,2008,1,1,385,...,43.0,1,0,01,1,5880,0,43.0,385,1


In [229]:
tumortrain, tumortest = train_test_split(newtumor, test_size=0.375)

In [230]:
tumortrain

,form_completion_date,prior_glioma,gender,race,ethnicity,history_of_neoadjuvant_treatment,year_of_initial_pathologic_diagnosis,initial_pathologic_diagnosis_method,vital_status,days_to_last_followup,...,rad_duration,therapy_type,therapy_ongoing,number_cycles,regimen_indication_d,total_dose,Missing Count_d,drug_duration,event_days,performance_status
TCGA-06-0185,2008-12-8,0,0,1,0,0,2005,1,0,1126,...,7.0,0,0,24,0,689,0,658.0,1126,0
TCGA-06-0125,2008-12-8,0,1,1,0,0,2001,1,1,1439,...,3.0,1,0,14,1,300,0,424.0,1439,0
TCGA-14-1823,2009-9-1,1,1,1,0,0,2007,1,1,543,...,41.0,1,0,12,1,11150,0,430.0,543,1
TCGA-06-0184,2008-12-10,0,0,1,0,0,2005,1,0,1228,...,14.0,1,0,12,0,160,0,335.0,1228,0
TCGA-06-0188,2008-12-9,0,0,1,0,0,2006,1,0,866,...,43.0,1,0,08,0,254,0,265.0,866,0


In [231]:
tumortrain.to_csv(r'C:\Users\danah\Documents\FIXED Tumor Training Set.csv') #exported for analysis once I checked output
tumortest.to_csv(r'C:\Users\danah\Documents\FIXED Tumor Testing Set.csv') #exported for analysis once I checked output
fulltrain.to_csv(r'C:\Users\danah\Documents\FIXED Full Cox Training Set.csv') #exported for analysis once I checked output
fulltest.to_csv(r'C:\Users\danah\Documents\FIXED Full Cox Testing Set.csv') #exported for analysis once I checked output